<a href="https://colab.research.google.com/github/OrbitPortal/ARINC424_Converter/blob/master/ARINC424_Converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
input = '''
SPACD        HME   RJ111220VDUW N35334434E139454014    N35334434E139454014W0070000632  544WGEHANEDA (TOKYO)                201861302
SPACD        TLD   RJ111600 DH                     TLD N36011558E140122072     001362     WGEAMI                           208311702
SPACD RJTTRJ ITA   RJ111155 IT N                   ITA N35331597E139462922     000310  315WGETOKYO (HANEDA) INTL           213082002
SPACDB       TS    RJ103325H  W N34074100E134362200                       W0070           WGETOKUSHIMA                     221510810
SPACEAENRT   AKAGI RJ1    C DRB N36232833E139415628                       W0079     WGE           AKAGI                    224611503
SPACER       Y566        3110JYOGARJEA0E    RBFXX                     251001560000 FL150     UNLTD                         340921908
SPACER       Y566        3120LAXASRJEA0EE   RBF                       000000002510                                         340931908
SPACP RJTTRJAHND     110000110Y N35331200E139465200W007000021250HME RJ1400014000CI00NMWGE    TOKYO (HANEDA) INTL           675801413
SPACP RJTTRJCT6R76 RJ1    W F   N35181092E139490476                       W0075     WGE           T6R76                    679292002
SPACP RJTTRJCTAURA RJ1    W D   N35184611E139444734                       W0075     WGE           TAURA                    679311710
SPACP RJTTRJFR23   H      010KAIHORJEA1E  I    IF                                 + 06000     14000                 B FS   709941201
SPACP RJTTRJFR23   H      020TT301RJPC1E  F 010TF                     04600122      04000                 RW23  RJPGB FS   709971201
SPACP RJTTRJFR23   H      021TT302RJPC1E    302TF                     04600009      03715             -300          B FS   709991212
SPACP RJTTRJFR23   H      023TT303RJPC1E   L302RF       003100            0070      01490          165-300TTRF1 RJPCB-FS   710011713
SPACP RJTTRJFR23   H      025TT304RJPC1E    302TF                     27700012      01119             -300          B FS   710031212
SPACP RJTTRJFR23   H      027TT305RJPC1E   L302RF       002000            0017      00585             -300TTRF2 RJPCB FS   710051212
SPACP RJTTRJFR23   H      030RW23 RJPG1GY M 302TF                     23000015      00106             -300          B FS   710071201
SPACP RJTTRJFR23   H      040TT306RJPC1E M  010TF                                                                   B FS   710101201
SPACP RJTTRJFR23   H      050TT307RJPC1E    010TF                                                                   B FS   710121201
SPACP RJTTRJFR23   H      060UTIBORJEA1E    010TF                                   04500                           B FS   710141503
SPACP RJTTRJFR23   H      070UTIBORJEA1EE HL   HM                     35700054                     230              B-FS   710161713
SPACP RJTTRJGRW16R   1098431568 N35332247E139461819         +0040900016157553197 ITA 10000                                 712382002
SPACP RJTTRJGRW23    1082022295 N35322615E139491961               00055000051197 ITD 10000ITL L                            712421011
SPACP RJTTRJGRW34L   1098433368 N35321176E139470841         +0041500018000054197 IHA 10000                                 712441011
SPACP RJTTRJIITA 1   011155RW16RN35320629E1394712301570N35331514E1394628930640 10700500300W00805300016                     719092002
'''

with open('0-test.dat', 'wt') as f:
  print(input, file=f)

In [0]:
import re
import pandas
import sqlite3
import string

as_grid = r"^(S|T)[ ]{3}(A)(S).{7}(.)(.{2})(.)(.{3})(.).{9}(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3})(.{3}).{3}(.*)(.{4})$"
vor = r"^(S|T)(.{3})(D)( )(.{4})(.{2}).(.{4}).{2}(.{2})(0|1)(.{5})(.{5})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{4})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{5})(.{5})(.)(.{2})(.{3})(.{3})(.{25}).{3}(.)(.)(.*)(.{4})$"
db = r"^(S|T)(.{3})(D)(B)(.{4})(.{2}).(.{4}).{2}(.{2})(0|1)(.{5})(.{5})(N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2}).{23}(E|W)(.{4}).{11}(.{3})(.{30})(.*)(.{4})$"
ea = r"^(S|T)(.{3})(E)(A)(.{4})(.{2})( )(.{5})[ ](.{2})(0|1)[ ]{4}(.{3})(.{2})[ ](N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2})[ ]{23}(E|W)(.{4})[ ]{5}(.{3})[ ]{11}(.{25})(.*)(.{4})$"
ep = r"^(S|T)(.{3})(E)(P)(.{4})(.{2}).{15}(.{2})(.{5})(.{2})(.)(.)(0|1)(.{4})(.)(.{3})(.{2})(.{5})(.{5})(.{3})(.{3})(.{6})(.{3})(.{3})(.{3}).{18}(.{25})(.*)(.{4})$"
er = r"^(S|T)(.{3})(E)(R).{7}(.{5})..{6}(.{4})(.{5})(.{2})(.{1})(.{1})(0|1)(.{4})(.)(.)(.)(.)(..)(.)(.{4})(.{2})(.{3}).{3}(.{4})(.{4})(.{4})(.{4})(.{4}).(.{5})(.{5})(.{5})(.{3})(.{3})(.{3})(.{3}).{4}(.{2}).{4}.{3}(.*)(.{4})$"
pa = r"^(S|T)(.{3})(P).(.{4})(.{2})(A)(.{3}).{2}.{3}(0|1)(.{5})(.{3})(.)(.)(N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2})(E|W)(.{4})(.{5})(.{3})(.{4})(.{2})(.{5})(.{5})(.)(.{3})(.)(.)(.{3}).{4}(.{30})(.*)(.{4})$"
pc = r"^(S|T)(.{3})(P)( )(.{4})(.{2})(C)(.{5})[ ](.{2})(0|1)[ ]{4}(.{3})(.{2})[ ](N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2})[ ]{23}(E|W)(.{4})[ ]{5}(.{3})[ ]{11}(.{25})(.*)(.{4})$"
pdef = r"^(S|T)(.{3})(P).(.{4})(.{2})(D|E|F)(.{6})(.)(.{5}).(.{3})(.{5})(.{2})(.)(.)(0|1)(.{4})(.)(.{3})(.{2})(.)(.{4})(.{2})(.{6})(.{4})(.{4})(.{4})(.{4})(.)(.).{2}(.)(.)(.{5})(.{5})(.{5})(.{3})(.{4})(.{5})(.)(.{2})(.)(.)(.)(.)(.)(.).{3}(.*)(.{4})$"
pg = r"^(S|T)(.{3})(P).(.{4})(.{2})(G)(.{5}).{3}(0|1)(.{5})(.{4}).(N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2})(.)(.{4}).{4}(.{6})(.{5})(.{4})(.{2})(.{3})(.)(.{4})(.)(.{4})(.{4})(.).{6}(.{22})(.*)(.{4})$"
pi = r"^(S|T)(.{3})(P).(.{4})(.{2})(I)(.{4})(.).{3}(0|1)(.{5})(.{5})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{4})(N|S| )(.{2})(.{2})(.{2})(.{2})(E|W| )(.{3})(.{2})(.{2})(.{2})(.{4})(.)(.{4})(.{4})(.{3})(E|W)(.{4})(.{2})(.{5})(.{4})(.{2})(.)(.).{13}(.*)(.{4})$"
pn = r"^(S|T)(.{3})(P)(N)(.{4})(.{2}).(.{4}).{2}(.{2})(0|1)(.{5})(.{5})(N|S)(.{2})(.{2})(.{2})(.{2})(E|W)(.{3})(.{2})(.{2})(.{2}).{23}(E|W)(.{4}).{11}(.{3})(.{30})(.*)(.{4})$"
tc = r"^(S|T)[ ]{3}(T)(C)(.{2})(.)[ ]{19}(.{4})(.{4})(M|T)[ ]{2}(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5})(.{5}).{24}(.*)(.{4})$"

new_database = True
write_database = True

sinonav = False
add_tailored = True

FSLabs = False
JeeHell = False
MJC = True

with open('0-test.dat', 'r', encoding='ISO-8859-1') as ARINC424: 
  # FSLabs (Finished)
  if FSLabs:
    mora_fsl = []
    vhf_fsl = []
    ndb_fsl = []
    hold_fsl = []
    awy_fsl = []
    arpt_fsl = []
    wpt_fsl = []
    proc_fsl = []
    rwy_fsl = []
    loc_fsl = []
    crz_fsl = []
    MORA_RECD_NR = 0
    VHF_RECD_NR = 0
    NDB_RECD_NR = 0
    WPT_RECD_NR = 0
    HLD_RECD_NR = 0
    AWY_RECD_NR = 0
    ARPT_RECD_NR = 0
    RWY_RECD_NR = 0
    LOC_RECD_NR = 0
    CRZ_RECD_NR = 0
    for line in ARINC424: 
      mora_se = re.search(as_grid, line)
      vhf_se = re.search(vor, line)
      db_se = re.search(db, line)
      ea_se = re.search(ea, line)
      hold_se = re.search(ep, line)
      awy_se = re.search(er, line)
      arpt_se = re.search(pa, line)
      pc_se = re.search(pc, line)
      proc_se = re.search(pdef, line)
      rwy_se = re.search(pg, line)
      loc_se = re.search(pi, line)
      pn_se = re.search(pn, line)
      crz_se = re.search(tc, line)
      if mora_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = mora_se.group(1)
        SECT_CODE = mora_se.group(2)
        SUBS_CODE = mora_se.group(3)
        if mora_se.group(4) == "N":
          MORA_LAT = float(mora_se.group(5))
        else:
          MORA_LAT = 0 - float(mora_se.group(5))
        if mora_se.group(6) == "E":
          MORA_LON = float(mora_se.group(7))
        else:
          MORA_LON = 0 - float(mora_se.group(7))
        MORA_TYPE = mora_se.group(8)
        if mora_se.group(9) == "UNK":
          MORA_00  = None
        else:
          MORA_00 = mora_se.group(9)
        if mora_se.group(10) == "UNK":
          MORA_01  = None
        else:
          MORA_01 = mora_se.group(10)
        if mora_se.group(11) == "UNK":
          MORA_02  = None
        else:
          MORA_02 = mora_se.group(11)
        if mora_se.group(12) == "UNK":
          MORA_03  = None
        else:
          MORA_03 = mora_se.group(12)
        if mora_se.group(13) == "UNK":
          MORA_04  = None
        else:
          MORA_04 = mora_se.group(13)
        if mora_se.group(14) == "UNK":
          MORA_05  = None
        else:
          MORA_05 = mora_se.group(14)
        if mora_se.group(15) == "UNK":
          MORA_06  = None
        else:
          MORA_06 = mora_se.group(15)
        if mora_se.group(16) == "UNK":
          MORA_07  = None
        else:
          MORA_07 = mora_se.group(16)
        if mora_se.group(17) == "UNK":
          MORA_08  = None
        else:
          MORA_08 = mora_se.group(17)
        if mora_se.group(18) == "UNK":
          MORA_09  = None
        else:
          MORA_09 = mora_se.group(18)
        if mora_se.group(19) == "UNK":
          MORA_10  = None
        else:
          MORA_10 = mora_se.group(19)
        if mora_se.group(20) == "UNK":
          MORA_11  = None
        else:
          MORA_11 = mora_se.group(20)
        if mora_se.group(21) == "UNK":
          MORA_12  = None
        else:
          MORA_12 = mora_se.group(21)
        if mora_se.group(22) == "UNK":
          MORA_13  = None
        else:
          MORA_13 = mora_se.group(22)
        if mora_se.group(23) == "UNK":
          MORA_14  = None
        else:
          MORA_14 = mora_se.group(23)
        if mora_se.group(24) == "UNK":
          MORA_15  = None
        else:
          MORA_15 = mora_se.group(24)
        if mora_se.group(25) == "UNK":
          MORA_16  = None
        else:
          MORA_16 = mora_se.group(25)
        if mora_se.group(26) == "UNK":
          MORA_17  = None
        else:
          MORA_17 = mora_se.group(26)
        if mora_se.group(27) == "UNK":
          MORA_18  = None
        else:
          MORA_18 = mora_se.group(27)
        if mora_se.group(28) == "UNK":
          MORA_19  = None
        else:
          MORA_19 = mora_se.group(28)
        if mora_se.group(29) == "UNK":
          MORA_20  = None
        else:
          MORA_20 = mora_se.group(29)
        if mora_se.group(30) == "UNK":
          MORA_21  = None
        else:
          MORA_21 = mora_se.group(30)
        if mora_se.group(31) == "UNK":
          MORA_22  = None
        else:
          MORA_22 = mora_se.group(31)
        if mora_se.group(32) == "UNK":
          MORA_23  = None
        else:
          MORA_23 = mora_se.group(32)
        if mora_se.group(33) == "UNK":
          MORA_24  = None
        else:
          MORA_24 = mora_se.group(33)
        if mora_se.group(34) == "UNK":
          MORA_25  = None
        else:
          MORA_25 = mora_se.group(34)
        if mora_se.group(35) == "UNK":
          MORA_26  = None
        else:
          MORA_26 = mora_se.group(35)
        if mora_se.group(36) == "UNK":
          MORA_27  = None
        else:
          MORA_27 = mora_se.group(36)
        if mora_se.group(37) == "UNK":
          MORA_28  = None
        else:
          MORA_28 = mora_se.group(37)
        if mora_se.group(38) == "UNK":
          MORA_29  = None
        else:
          MORA_29 = mora_se.group(38)
        MORA_RECD_NR = MORA_RECD_NR + 1
        CYCLE = mora_se.group(40)
        mora_fsl.append((RECD_TYPE,SECT_CODE,SUBS_CODE,MORA_LAT,MORA_LON,MORA_TYPE,MORA_00,MORA_01,MORA_02,MORA_03,MORA_04,MORA_05,MORA_06,MORA_07,MORA_08,MORA_09,MORA_10,MORA_11,MORA_12,MORA_13,MORA_14,MORA_15,MORA_16,MORA_17,MORA_18,MORA_19,MORA_20,MORA_21,MORA_22,MORA_23,MORA_24,MORA_25,MORA_26,MORA_27,MORA_28,MORA_29,MORA_RECD_NR,CYCLE))
      if vhf_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = vhf_se.group(1)
        CUST_AREA = vhf_se.group(2)
        SECT_CODE = vhf_se.group(3)
        SUBS_CODE = vhf_se.group(4)
        ARPT_IDENT = vhf_se.group(5).strip()
        ARPT_ICAO_CODE = vhf_se.group(6)
        VOR_IDENT = vhf_se.group(7).strip()
        VHF_ICAO_CODE = vhf_se.group(8)
        CONT_NR = vhf_se.group(9)
        VOR_FREQ = float(vhf_se.group(10))/100
        NAVAID_CLASS = vhf_se.group(11)
        if vhf_se.group(12).strip() == "":
          VOR_LAT = None
        elif vhf_se.group(12) == "N":
          VOR_LAT = float(vhf_se.group(13)) + float(vhf_se.group(14))/60 + float(vhf_se.group(15))/3600 + float(vhf_se.group(16))/360000
        else:
          VOR_LAT = 0 - float(vhf_se.group(13)) - float(vhf_se.group(14))/60 - float(vhf_se.group(15))/3600 - float(vhf_se.group(16))/360000
        if vhf_se.group(17).strip() == "":
          VOR_LON = None
        elif vhf_se.group(17) == "E":
          VOR_LON = float(vhf_se.group(18)) + float(vhf_se.group(19))/60 + float(vhf_se.group(20))/3600 + float(vhf_se.group(21))/360000
        else:
          VOR_LON = 0 - float(vhf_se.group(18)) - float(vhf_se.group(19))/60 - float(vhf_se.group(20))/3600 - float(vhf_se.group(21))/360000
        DME_IDENT = vhf_se.group(22).strip()
        if vhf_se.group(23).strip() == "":
          DME_LAT = None
        elif vhf_se.group(23) == "N":
          DME_LAT = float(vhf_se.group(24)) + float(vhf_se.group(25))/60 + float(vhf_se.group(26))/3600 + float(vhf_se.group(27))/360000
        else:
          DME_LAT = 0 - float(vhf_se.group(24)) - float(vhf_se.group(25))/60 - float(vhf_se.group(26))/3600 - float(vhf_se.group(27))/360000
        if vhf_se.group(28).strip() == "":
          DME_LON = None
        elif vhf_se.group(28) == "E":
          DME_LON = float(vhf_se.group(29)) + float(vhf_se.group(30))/60 + float(vhf_se.group(31))/3600 + float(vhf_se.group(32))/360000
        else:
          DME_LON = 0 - float(vhf_se.group(29)) - float(vhf_se.group(30))/60 - float(vhf_se.group(31))/3600 - float(vhf_se.group(32))/360000
        STATION_DECLINATION = vhf_se.group(33)
        if vhf_se.group(34).strip() == "":
          DME_ELEV = None
        else:
          DME_ELEV = vhf_se.group(34)
        FIGURE_OF_MERIT = vhf_se.group(35)
        if vhf_se.group(36).strip() == "":
          ILS_DME_BIAS = None
        else:
          ILS_DME_BIAS = vhf_se.group(36)
        FREQ_PROTECTION = vhf_se.group(37)
        DATUM_CODE = vhf_se.group(38)
        VOR_NAME = vhf_se.group(39).replace('VOR/DME','       ')
        ROUTE_INAPPROPRIATE_DME = vhf_se.group(40)
        DME_OPERATIONAL_SERVICE_VOLUME = vhf_se.group(41)
        VHF_RECD_NR = VHF_RECD_NR + 1
        CYCLE = vhf_se.group(43)
        vhf_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,ARPT_IDENT,ARPT_ICAO_CODE,VOR_IDENT,VHF_ICAO_CODE,CONT_NR,VOR_FREQ,NAVAID_CLASS,VOR_LAT,VOR_LON,DME_IDENT,DME_LAT,DME_LON,STATION_DECLINATION,DME_ELEV,FIGURE_OF_MERIT,ILS_DME_BIAS,FREQ_PROTECTION,DATUM_CODE,VOR_NAME,ROUTE_INAPPROPRIATE_DME,DME_OPERATIONAL_SERVICE_VOLUME,VHF_RECD_NR,CYCLE))
      if db_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = db_se.group(1)
        CUST_AREA = db_se.group(2)
        SECT_CODE = db_se.group(3)
        SUBS_CODE = db_se.group(4)
        ARPT_IDENT = db_se.group(5).strip()
        ARPT_ICAO_CODE = db_se.group(6)
        NDB_IDENT = db_se.group(7).strip()
        NDB_ICAO_CODE = db_se.group(8)
        CONT_NR = db_se.group(9)
        NDB_FREQ = float(db_se.group(10))/10
        NDB_CLASS = db_se.group(11)
        if db_se.group(12) == "N":
          NDB_LAT = float(db_se.group(13)) + float(db_se.group(14))/60 + float(db_se.group(15))/3600 + float(db_se.group(16))/360000
        else:
          NDB_LAT = 0 - float(db_se.group(13)) - float(db_se.group(14))/60 - float(db_se.group(15))/3600 - float(db_se.group(16))/360000
        if db_se.group(17) == "E":
          NDB_LON = float(db_se.group(18)) + float(db_se.group(19))/60 + float(db_se.group(20))/3600 + float(db_se.group(21))/360000
        else:
          NDB_LON = 0 - float(db_se.group(18)) - float(db_se.group(19))/60 - float(db_se.group(20))/3600 - float(db_se.group(21))/360000
        if db_se.group(22) == "E":
          MAG_VAR = float(db_se.group(23))/100
        else:
          MAG_VAR = 0 - float(db_se.group(23))/100
        DATUM_CODE = db_se.group(24)
        NDB_NAME = db_se.group(25).replace('NDB','   ')
        NDB_RECD_NR = NDB_RECD_NR + 1
        CYCLE = db_se.group(27)      
        ndb_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,ARPT_IDENT,ARPT_ICAO_CODE,NDB_IDENT,NDB_ICAO_CODE,CONT_NR,NDB_FREQ,NDB_CLASS,NDB_LAT,NDB_LON,MAG_VAR,DATUM_CODE,NDB_NAME,NDB_RECD_NR,CYCLE))
      if ea_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = ea_se.group(1)
        CUST_AREA = ea_se.group(2)
        SECT_CODE = ea_se.group(3)
        SUBS_CODE = ea_se.group(4)
        REGION_CODE = ea_se.group(5)
        REGION_ICAO_CODE = ea_se.group(6)
        REGION_SUBS_CODE = ea_se.group(7)
        WAYPOINT_IDENT = ea_se.group(8).strip()
        WAYPOINT_ICAO_CODE = ea_se.group(9)
        CONT_NR = ea_se.group(10)
        WAYPOINT_TYPE = ea_se.group(11)
        WAYPOINT_USAGE = ea_se.group(12)
        if ea_se.group(13) == "N":
          WAYPOINT_LAT = float(ea_se.group(14)) + float(ea_se.group(15))/60 + float(ea_se.group(16))/3600 + float(ea_se.group(17))/360000
        else:
          WAYPOINT_LAT = 0 - float(ea_se.group(14)) - float(ea_se.group(15))/60 - float(ea_se.group(16))/3600 - float(ea_se.group(17))/360000
        if ea_se.group(18) == "E":
          WAYPOINT_LON = float(ea_se.group(19)) + float(ea_se.group(20))/60 + float(ea_se.group(21))/3600 + float(ea_se.group(22))/360000
        else:
          WAYPOINT_LON = 0 - float(ea_se.group(19)) - float(ea_se.group(20))/60 - float(ea_se.group(21))/3600 - float(ea_se.group(22))/360000
        if ea_se.group(23) == "E":
          DYNAMIC_MAG_VAR = float(ea_se.group(24))/100
        else:
          DYNAMIC_MAG_VAR = 0 - float(ea_se.group(24))/100
        DATUM_CODE = ea_se.group(25)
        WAYPOINT_NAME_DESCR = ea_se.group(26)
        WPT_RECD_NR = WPT_RECD_NR + 1
        CYCLE = ea_se.group(28)
        wpt_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,REGION_CODE,REGION_ICAO_CODE,REGION_SUBS_CODE,WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,CONT_NR,WAYPOINT_TYPE,WAYPOINT_USAGE,WAYPOINT_LAT,WAYPOINT_LON,DYNAMIC_MAG_VAR,DATUM_CODE,"   ",WAYPOINT_NAME_DESCR,WPT_RECD_NR,CYCLE))
      if hold_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = hold_se.group(1)
        CUST_AREA = hold_se.group(2)
        SECT_CODE = hold_se.group(3)
        SUBS_CODE = hold_se.group(4)
        REGION_CODE = hold_se.group(5)
        REGION_ICAO_CODE = hold_se.group(6)
        DUPLICATE_IDENT = hold_se.group(7)
        FIX_IDENT = hold_se.group(8).strip()
        FIX_ICAO_CODE = hold_se.group(9)
        FIX_SECT_CODE = hold_se.group(10)
        FIX_SUBS_CODE = hold_se.group(11)
        CONT_NR = hold_se.group(12)
        if hold_se.group(13).lstrip(string.digits) == "T":
          INBOUND_HOLDING_COURSE = hold_se.group(13).rstrip('T')
        else:
          INBOUND_HOLDING_COURSE = float(hold_se.group(13))//10
        TURN_DIR = hold_se.group(14)
        if hold_se.group(15).strip() == "":
          LEG_LENGTH = None
        else:
          LEG_LENGTH = float(hold_se.group(15))//10
        LEG_TIME = hold_se.group(16)
        if hold_se.group(17).strip() == "":
          MINIMUM_ALT = None
        elif hold_se.group(17).strip() == "NESTB":
          MINIMUM_ALT = None
        elif hold_se.group(17).rstrip(string.digits) == "FL":
          MINIMUM_ALT = hold_se.group(17).lstrip(string.ascii_uppercase) + "00"
        else:
          MINIMUM_ALT = hold_se.group(17)
        if hold_se.group(18).strip() == "":
          MAXIMUM_ALT = None
        elif hold_se.group(18).strip() == "UNLTD":
          MAXIMUM_ALT = None
        elif hold_se.group(18).rstrip(string.digits) == "FL":
          MAXIMUM_ALT = hold_se.group(18).lstrip(string.ascii_uppercase) + "00"
        else:
          MAXIMUM_ALT = hold_se.group(18)
        if hold_se.group(19).strip() == "":
          HOLDING_SPEED = None
        else:
          HOLDING_SPEED = hold_se.group(19)
        if hold_se.group(20).strip() == "":
          RNP = None
        else:
          RNP = hold_se.group(20)
        if hold_se.group(21).strip() == "":
          ARC_RADIUS = None
        else:
          ARC_RADIUS = hold_se.group(21).strip().lstrip('0')
        VERTICAL_SCALE_FACTOR = hold_se.group(22)
        if hold_se.group(23).strip() == "":
          RVSM_MINIMUM_LEVEL = None
        else:
          RVSM_MINIMUM_LEVEL = hold_se.group(23)
        if hold_se.group(24).strip() == "":
          RVSM_MAXIMUM_LEVEL = None
        else:
          RVSM_MAXIMUM_LEVEL = hold_se.group(24)
        NAME = hold_se.group(25)
        HLD_RECD_NR = HLD_RECD_NR + 1
        CYCLE = hold_se.group(27)
        hold_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,REGION_CODE,REGION_ICAO_CODE,DUPLICATE_IDENT,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_CODE,FIX_SUBS_CODE,CONT_NR,INBOUND_HOLDING_COURSE,TURN_DIR,LEG_LENGTH,LEG_TIME,MINIMUM_ALT,MAXIMUM_ALT,HOLDING_SPEED,RNP,ARC_RADIUS,VERTICAL_SCALE_FACTOR,RVSM_MINIMUM_LEVEL,RVSM_MAXIMUM_LEVEL,NAME,HLD_RECD_NR,CYCLE))
      if awy_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = awy_se.group(1)
        CUST_AREA = awy_se.group(2)
        SECT_CODE = awy_se.group(3)
        SUBS_CODE = awy_se.group(4)
        ROUTE_IDENT = awy_se.group(5).strip()
        SEQ_NR = awy_se.group(6)
        FIX_IDENT = awy_se.group(7).strip()
        FIX_ICAO_CODE = awy_se.group(8)
        FIX_SECT_CODE = awy_se.group(9)
        FIX_SUBS_CODE = awy_se.group(10)
        CONT_NR = awy_se.group(11)
        WAYPOINT_DESCR_CODE = awy_se.group(12)
        BOUNDARY_CODE = awy_se.group(13)
        ROUTE_TYPE = awy_se.group(14)
        LEVEL = awy_se.group(15)
        DIRECTION_RESTRICTION = awy_se.group(16)
        CRUISE_TABLE_IND = awy_se.group(17)
        EU_IND = awy_se.group(18)
        RECOMMENDED_NAVAID = awy_se.group(19)
        RECOMMENDED_NAVAID_ICAO_CODE = awy_se.group(20)
        if awy_se.group(21).strip() == "":
          RNP = None
        else:
          RNP = awy_se.group(21)
        if awy_se.group(22).strip() == "":
          THETA = None
        else:
          THETA = awy_se.group(22)
        if awy_se.group(23).strip() == "":
          RHO = None
        else:
          RHO = awy_se.group(23)
        if awy_se.group(24).strip() == "":
          OUTBOUND_MAG_COURSE = None
        elif awy_se.group(24).lstrip(string.digits) == "T":
          OUTBOUND_MAG_COURSE = awy_se.group(24).rstrip('T')
        else:
          OUTBOUND_MAG_COURSE = float(awy_se.group(24))//100
        ROUTE_DISTANCE_FROM = awy_se.group(25)
        if awy_se.group(26).strip() == "":
          INBOUND_MAG_COURSE = None
        elif awy_se.group(26).lstrip(string.digits) == "T":
          INBOUND_MAG_COURSE = awy_se.group(26).rstrip('T')
        else:
          INBOUND_MAG_COURSE = float(awy_se.group(26))//100
        if awy_se.group(27).strip() == "":
          MINIMUM_ALT = None
        elif awy_se.group(27).strip() == "NESTB":
          MINIMUM_ALT = None
        elif awy_se.group(27).rstrip(string.digits) == "FL":
          MINIMUM_ALT = awy_se.group(27).lstrip(string.ascii_uppercase) + "00"
        else:
          MINIMUM_ALT = awy_se.group(27)
        if awy_se.group(28).strip() == "":
          MINIMUM_ALT_1 = None
        elif awy_se.group(28).strip() == "NESTB":
          MINIMUM_ALT_1 = None
        elif awy_se.group(28).rstrip(string.digits) == "FL":
          MINIMUM_ALT_1 = awy_se.group(28).lstrip(string.ascii_uppercase) + "00"
        else:
          MINIMUM_ALT_1 = awy_se.group(28)
        if awy_se.group(29).strip() == "":
          MAXIMUM_ALT = None
        elif awy_se.group(29).strip() == "UNLTD":
          MAXIMUM_ALT = None
        elif awy_se.group(29).rstrip(string.digits) == "FL":
          MAXIMUM_ALT = awy_se.group(29).lstrip(string.ascii_uppercase) + "00"
        else:
          MAXIMUM_ALT = awy_se.group(29)
        FIX_RADIUS_TRANSITION_IND = awy_se.group(30)
        VERTICAL_SCALE_FACTOR = awy_se.group(31)
        if awy_se.group(32).strip() == "":
          RVSM_MINIMUM_LEVEL = None
        else:
          RVSM_MINIMUM_LEVEL = awy_se.group(32)
        if awy_se.group(33).strip() == "":
          VSF_RVSM_MAXIMUM_LEVEL = None
        else:
          VSF_RVSM_MAXIMUM_LEVEL = awy_se.group(33)
        ICAO_CODE_2 = awy_se.group(34)
        AWY_RECD_NR = AWY_RECD_NR + 1
        CYCLE = awy_se.group(36)
        awy_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,ROUTE_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_CODE,FIX_SUBS_CODE,CONT_NR,WAYPOINT_DESCR_CODE,BOUNDARY_CODE,ROUTE_TYPE,LEVEL,DIRECTION_RESTRICTION,CRUISE_TABLE_IND,EU_IND,RECOMMENDED_NAVAID,RECOMMENDED_NAVAID_ICAO_CODE,RNP,THETA,RHO,OUTBOUND_MAG_COURSE,ROUTE_DISTANCE_FROM,INBOUND_MAG_COURSE,MINIMUM_ALT,MINIMUM_ALT_1,MAXIMUM_ALT,FIX_RADIUS_TRANSITION_IND,VERTICAL_SCALE_FACTOR,RVSM_MINIMUM_LEVEL,VSF_RVSM_MAXIMUM_LEVEL,ICAO_CODE_2,AWY_RECD_NR,CYCLE))
      if arpt_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = arpt_se.group(1)
        CUST_AREA = arpt_se.group(2)
        SECT_CODE = arpt_se.group(3)
        ARPT_IDENT = arpt_se.group(4)
        ARPT_ICAO_CODE = arpt_se.group(5)
        SUBS_CODE = arpt_se.group(6)
        IATA_IDENT = arpt_se.group(7).strip()
        CONT_NR = arpt_se.group(8)
        if arpt_se.group(9).strip() == "":
          SPEED_LIMIT_ALT = None
        else:
          SPEED_LIMIT_ALT = arpt_se.group(9)
        LONGEST_RUNWAY = arpt_se.group(10)
        IFR_CAPABILITY = arpt_se.group(11)
        LONGEST_RUNWAY_SURFACE_CODE = arpt_se.group(12)
        if arpt_se.group(13) == "N":
          ARPT_LAT = float(arpt_se.group(14)) + float(arpt_se.group(15))/60 + float(arpt_se.group(16))/3600 + float(arpt_se.group(17))/360000
        else:
          ARPT_LAT = 0 - float(arpt_se.group(14)) - float(arpt_se.group(15))/60 - float(arpt_se.group(16))/3600 - float(arpt_se.group(17))/360000
        if arpt_se.group(18) == "E":
          ARPT_LON = float(arpt_se.group(19)) + float(arpt_se.group(20))/60 + float(arpt_se.group(21))/3600 + float(arpt_se.group(22))/360000
        else:
          ARPT_LON = 0 - float(arpt_se.group(19)) - float(arpt_se.group(20))/60 - float(arpt_se.group(21))/3600 - float(arpt_se.group(22))/360000
        if arpt_se.group(23) == "E":
          MAG_VAR = float(arpt_se.group(24))/100
        else:
          MAG_VAR = 0 - float(arpt_se.group(24))/100
        ARPT_ELEV = arpt_se.group(25)
        if arpt_se.group(26).strip() == "":
          SPEED_LIMIT = None
        else:
          SPEED_LIMIT = arpt_se.group(26)
        RECOMMENDED_NAVAID = arpt_se.group(27)
        RECOMMENDED_NAVAID_ICAO_CODE = arpt_se.group(28)
        if arpt_se.group(29).strip() == "":
          TRANSITIONS_ALT = None
        else:
          TRANSITIONS_ALT = arpt_se.group(29)
        if arpt_se.group(30).strip() == "":
          TRANSITION_LEVEL = None
        else:
          TRANSITION_LEVEL = arpt_se.group(30)
        PUBLIC_MILITARY_IND = arpt_se.group(31)
        TIME_ZONE = arpt_se.group(32)
        DAYLIGHT_IND = arpt_se.group(33)
        MAG_TRUE_IND = arpt_se.group(34)
        DATUM_CODE = arpt_se.group(35)
        ARPT_NAME = arpt_se.group(36)
        ARPT_RECD_NR = ARPT_RECD_NR + 1
        CYCLE = arpt_se.group(38)
        arpt_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,ARPT_IDENT,ARPT_ICAO_CODE,SUBS_CODE,IATA_IDENT,CONT_NR,SPEED_LIMIT_ALT,LONGEST_RUNWAY,IFR_CAPABILITY,LONGEST_RUNWAY_SURFACE_CODE,ARPT_LAT,ARPT_LON,MAG_VAR,ARPT_ELEV,SPEED_LIMIT,RECOMMENDED_NAVAID,RECOMMENDED_NAVAID_ICAO_CODE,TRANSITIONS_ALT,TRANSITION_LEVEL,PUBLIC_MILITARY_IND,TIME_ZONE,DAYLIGHT_IND,MAG_TRUE_IND,DATUM_CODE,ARPT_NAME,ARPT_RECD_NR,CYCLE))
      if pc_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = pc_se.group(1)
        CUST_AREA = pc_se.group(2)
        SECT_CODE = pc_se.group(3)
        SUBS_CODE = pc_se.group(4)
        REGION_CODE = pc_se.group(5)
        REGION_ICAO_CODE = pc_se.group(6)
        REGION_SUBS_CODE = pc_se.group(7)
        WAYPOINT_IDENT = pc_se.group(8).strip()
        WAYPOINT_ICAO_CODE = pc_se.group(9)
        CONT_NR = pc_se.group(10)
        WAYPOINT_TYPE = pc_se.group(11)
        WAYPOINT_USAGE = pc_se.group(12)
        if pc_se.group(13) == "N":
          WAYPOINT_LAT = float(pc_se.group(14)) + float(pc_se.group(15))/60 + float(pc_se.group(16))/3600 + float(pc_se.group(17))/360000
        else:
          WAYPOINT_LAT = 0 - float(pc_se.group(14)) - float(pc_se.group(15))/60 - float(pc_se.group(16))/3600 - float(pc_se.group(17))/360000
        if pc_se.group(18) == "E":
          WAYPOINT_LON = float(pc_se.group(19)) + float(pc_se.group(20))/60 + float(pc_se.group(21))/3600 + float(pc_se.group(22))/360000
        else:
          WAYPOINT_LON = 0 - float(pc_se.group(19)) - float(pc_se.group(20))/60 - float(pc_se.group(21))/3600 - float(pc_se.group(22))/360000
        if pc_se.group(23) == "E":
          DYNAMIC_MAG_VAR = float(pc_se.group(24))/100
        else:
          DYNAMIC_MAG_VAR = 0 - float(pc_se.group(24))/100
        DATUM_CODE = pc_se.group(25)
        WAYPOINT_NAME_DESCR = pc_se.group(26)
        PC_RECD_NR = pc_se.group(27)
        CYCLE = pc_se.group(28)
        wpt_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,REGION_CODE,REGION_ICAO_CODE,REGION_SUBS_CODE,WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,CONT_NR,WAYPOINT_TYPE,WAYPOINT_USAGE,WAYPOINT_LAT,WAYPOINT_LON,DYNAMIC_MAG_VAR,DATUM_CODE,"   ",WAYPOINT_NAME_DESCR,PC_RECD_NR,CYCLE))
      if proc_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = proc_se.group(1)
        CUST_AREA = proc_se.group(2)
        SECT_CODE = proc_se.group(3)
        ARPT_IDENT = proc_se.group(4)
        ARPT_ICAO_CODE = proc_se.group(5)
        SUBS_CODE = proc_se.group(6)
        PROC_IDENT = proc_se.group(7).strip()
        ROUTE_TYPE = proc_se.group(8)
        TRANSITION_IDENT = proc_se.group(9).strip()
        SEQ_NR = proc_se.group(10)
        FIX_IDENT = proc_se.group(11).strip()
        FIX_ICAO_CODE = proc_se.group(12)
        FIX_SECT_CODE = proc_se.group(13)
        FIX_SUBS_CODE = proc_se.group(14)
        CONT_NR = proc_se.group(15)
        WAYPOINT_DESCR_CODE = proc_se.group(16)
        TURN_DIR = proc_se.group(17)
        if proc_se.group(18).strip() == "":
          RNP = None
        else:
          RNP = proc_se.group(18)
        PATH_AND_TERMINATION = proc_se.group(19)
        TURN_DIR_VALID = proc_se.group(20)
        RECOMMENDED_NAVAID = proc_se.group(21)
        RECOMMENDED_NAVAID_ICAO_CODE = proc_se.group(22)
        if proc_se.group(23).strip() == "":
          ARC_RADIUS = None
        else:
          ARC_RADIUS = proc_se.group(23).strip().lstrip('0')
        if proc_se.group(24).strip() == "":
          THETA = None
        else:
          THETA = proc_se.group(24)
        if proc_se.group(25).strip() == "":
          RHO = None
        else:
          RHO = proc_se.group(25)
        if proc_se.group(26).strip() == "":
          MAG_COURSE = None
        elif proc_se.group(26).lstrip(string.digits) == "T":
          MAG_COURSE = proc_se.group(26).rstrip('T')
        else:
          MAG_COURSE = float(proc_se.group(26))//10
        ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME = proc_se.group(27)
        RECOMMENDED_NAVAID_SECT_CODE = proc_se.group(28)
        RECOMMENDED_NAVAID_SUBS_CODE = proc_se.group(29)
        ALT_DESCR = proc_se.group(30)
        ATC_IND = proc_se.group(31)
        if proc_se.group(32).strip() == "":
          ALT_1 = None
        elif proc_se.group(32).rstrip(string.digits) == "FL":
          ALT_1 = proc_se.group(32).lstrip(string.ascii_uppercase) + "00"
        else:
          ALT_1 = proc_se.group(32)
        if proc_se.group(33).strip() == "":
          ALT_2 = None
        elif proc_se.group(33).rstrip(string.digits) == "FL":
          ALT_2 = proc_se.group(33).lstrip(string.ascii_uppercase) + "00"
        else:
          ALT_2 = proc_se.group(33)
        if proc_se.group(34).strip() == "":
          TRANSITION_ALT = None
        else:
          TRANSITION_ALT = proc_se.group(34)
        SPEED_LIMIT = proc_se.group(35)
        if proc_se.group(36).strip() == "":
          VERTICAL_ANGLE = None
        else:
          VERTICAL_ANGLE = float(proc_se.group(36))/100
        CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND = proc_se.group(37)
        MULTIPLE_CODE_OR_TAA_SECTOR_IDENT = proc_se.group(38).strip()
        MULTIPLE_CODE_OR_TAA_SECTOR_ICAO_CODE = proc_se.group(39)
        MULTIPLE_CODE_OR_TAA_SECTOR_SECT_CODE = proc_se.group(40)
        MULTIPLE_CODE_OR_TAA_SECTOR_SUBS_CODE = proc_se.group(41)
        GNSS_FMS_INDICATION = proc_se.group(42)
        SPEED_LIMIT_DESCR = proc_se.group(43)
        APCH_ROUTE_QUALIFIER_1 = proc_se.group(44)
        APCH_ROUTE_QUALIFIER_2 = proc_se.group(45)
        PROC_RECD_NR = proc_se.group(46)
        CYCLE = proc_se.group(47)
        proc_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,ARPT_IDENT,ARPT_ICAO_CODE,SUBS_CODE,PROC_IDENT,ROUTE_TYPE,TRANSITION_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_CODE,FIX_SUBS_CODE,CONT_NR,WAYPOINT_DESCR_CODE,TURN_DIR,RNP,PATH_AND_TERMINATION,TURN_DIR_VALID,RECOMMENDED_NAVAID,RECOMMENDED_NAVAID_ICAO_CODE,ARC_RADIUS,THETA,RHO,MAG_COURSE,ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME,RECOMMENDED_NAVAID_SECT_CODE,RECOMMENDED_NAVAID_SUBS_CODE,ALT_DESCR,ATC_IND,ALT_1,ALT_2,TRANSITION_ALT,SPEED_LIMIT,VERTICAL_ANGLE,CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND,MULTIPLE_CODE_OR_TAA_SECTOR_IDENT,MULTIPLE_CODE_OR_TAA_SECTOR_ICAO_CODE,MULTIPLE_CODE_OR_TAA_SECTOR_SECT_CODE,MULTIPLE_CODE_OR_TAA_SECTOR_SUBS_CODE,GNSS_FMS_INDICATION,SPEED_LIMIT_DESCR,APCH_ROUTE_QUALIFIER_1,APCH_ROUTE_QUALIFIER_2,"   ",PROC_RECD_NR,CYCLE))
      if rwy_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = rwy_se.group(1)
        CUST_AREA = rwy_se.group(2)
        SECT_CODE = rwy_se.group(3)
        ARPT_IDENT = rwy_se.group(4)
        ARPT_ICAO_CODE = rwy_se.group(5)
        SUBS_CODE = rwy_se.group(6)
        RUNWAY_IDENT = rwy_se.group(7).strip()
        CONT_NR = rwy_se.group(8)
        RUNWAY_LENGTH = rwy_se.group(9)
        if rwy_se.group(10).lstrip(string.digits) == "T":
          RUNWAY_MAG_BEARING = rwy_se.group(10).rstrip('T')
        else:
          RUNWAY_MAG_BEARING = float(rwy_se.group(10))//10
        if rwy_se.group(11) == "N":
          RUNWAY_LAT = float(rwy_se.group(12)) + float(rwy_se.group(13))/60 + float(rwy_se.group(14))/3600 + float(rwy_se.group(15))/360000
        else:
          RUNWAY_LAT = 0 - float(rwy_se.group(12)) - float(rwy_se.group(13))/60 - float(rwy_se.group(14))/3600 - float(rwy_se.group(15))/360000
        if rwy_se.group(16) == "E":
          RUNWAY_LON = float(rwy_se.group(17)) + float(rwy_se.group(18))/60 + float(rwy_se.group(19))/3600 + float(rwy_se.group(20))/360000
        else:
          RUNWAY_LON = 0 - float(rwy_se.group(17)) - float(rwy_se.group(18))/60 - float(rwy_se.group(19))/3600 - float(rwy_se.group(20))/360000      
        if rwy_se.group(21).strip() == "":
          RUNWAY_GRADIENT = None
        elif rwy_se.group(21) == "-":
          RUNWAY_GRADIENT = 0 - float(rwy_se.group(22))/100
        else:
          RUNWAY_GRADIENT = float(rwy_se.group(22))/100
        if rwy_se.group(23).strip() == "":
          ELLIPSOID_HEIGHT = None
        else:
          ELLIPSOID_HEIGHT = rwy_se.group(23)
        LANDING_THLD_ELEV = rwy_se.group(24)
        if rwy_se.group(25).strip() == "":
          DISPLACED_THLD_DISTANCE = "0"
        else:
          DISPLACED_THLD_DISTANCE = rwy_se.group(25)
        if rwy_se.group(26).strip() == "":
          THLD_CROSSING_HEIGHT = "50"
        else:
          THLD_CROSSING_HEIGHT = rwy_se.group(26)
        RUNWAY_WIDTH = rwy_se.group(27)
        TCH_VALUE_IND = rwy_se.group(28)
        LOCALIZER_MLS_GLS_REF_PATH_IDENT = rwy_se.group(29).strip()
        LOCALIZER_MLS_GLS_CATEGORY_CLASS = rwy_se.group(30)
        STOPWAY = rwy_se.group(31)
        SECOND_LOCALIZER_MLS_GLS_REF_PATH_IDENT = rwy_se.group(32).strip()
        SECOND_LOCALIZER_MLS_GLS_CATEGORY_CLASS = rwy_se.group(33)
        RUNWAY_DESCR = rwy_se.group(34)
        RWY_RECD_NR = RWY_RECD_NR + 1
        CYCLE = rwy_se.group(36)
        rwy_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,ARPT_IDENT,ARPT_ICAO_CODE,SUBS_CODE,RUNWAY_IDENT,CONT_NR,RUNWAY_LENGTH,RUNWAY_MAG_BEARING,RUNWAY_LAT,RUNWAY_LON,RUNWAY_GRADIENT,ELLIPSOID_HEIGHT,LANDING_THLD_ELEV,DISPLACED_THLD_DISTANCE,THLD_CROSSING_HEIGHT,RUNWAY_WIDTH,TCH_VALUE_IND,LOCALIZER_MLS_GLS_REF_PATH_IDENT,LOCALIZER_MLS_GLS_CATEGORY_CLASS,STOPWAY,SECOND_LOCALIZER_MLS_GLS_REF_PATH_IDENT,SECOND_LOCALIZER_MLS_GLS_CATEGORY_CLASS,RUNWAY_DESCR,RWY_RECD_NR,CYCLE))
      if loc_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = loc_se.group(1)
        CUST_AREA = loc_se.group(2)
        SECT_CODE = loc_se.group(3)
        ARPT_IDENT = loc_se.group(4)
        ARPT_ICAO_CODE = loc_se.group(5)
        SUBS_CODE = loc_se.group(6)
        LOCALIZER_IDENT = loc_se.group(7).strip()
        ILS_CATEGORY = loc_se.group(8)
        CONT_NR = loc_se.group(9)
        LOCALIZER_FREQ = float(loc_se.group(10))/100
        RUNWAY_IDENT = loc_se.group(11).strip()
        if loc_se.group(12).strip() == "":
          LOCALIZER_LAT = None
        elif loc_se.group(12) == "N":
          LOCALIZER_LAT = float(loc_se.group(13)) + float(loc_se.group(14))/60 + float(loc_se.group(15))/3600 + float(loc_se.group(16))/360000
        else:
          LOCALIZER_LAT = 0 - float(loc_se.group(13)) - float(loc_se.group(14))/60 - float(loc_se.group(15))/3600 - float(loc_se.group(16))/360000
        if loc_se.group(17).strip() == "":
          LOCALIZER_LON = None
        elif loc_se.group(17) == "E":
          LOCALIZER_LON = float(loc_se.group(18)) + float(loc_se.group(19))/60 + float(loc_se.group(20))/3600 + float(loc_se.group(21))/360000
        else:
          LOCALIZER_LON = 0 - float(loc_se.group(18)) - float(loc_se.group(19))/60 - float(loc_se.group(20))/3600 - float(loc_se.group(21))/360000
        if loc_se.group(22).lstrip(string.digits) == "T":
          LOCALIZER_BEARING = loc_se.group(22).rstrip('T')
        else:
          LOCALIZER_BEARING = float(loc_se.group(22))//10
        if loc_se.group(23).strip() == "":
          GLIDE_SLOPE_LAT = None
        elif loc_se.group(23) == "N":
          GLIDE_SLOPE_LAT = float(loc_se.group(24)) + float(loc_se.group(25))/60 + float(loc_se.group(26))/3600 + float(loc_se.group(27))/360000
        else:
          GLIDE_SLOPE_LAT = 0 - float(loc_se.group(24)) - float(loc_se.group(25))/60 - float(loc_se.group(26))/3600 - float(loc_se.group(27))/360000
        if loc_se.group(28).strip() == "":
          GLIDE_SLOPE_LON = None
        elif loc_se.group(28) == "E":
          GLIDE_SLOPE_LON = float(loc_se.group(29)) + float(loc_se.group(30))/60 + float(loc_se.group(31))/3600 + float(loc_se.group(32))/360000
        else:
          GLIDE_SLOPE_LON = 0 - float(loc_se.group(29)) - float(loc_se.group(30))/60 - float(loc_se.group(31))/3600 - float(loc_se.group(32))/360000
        LOCALIZER_POS = loc_se.group(33)
        LOCALIZER_POS_REFERENCE = loc_se.group(34)
        GLIDE_SLOPE_POS = loc_se.group(35)
        LOCALIZER_WIDTH = float(loc_se.group(36))
        if loc_se.group(37).strip() == "":
          GLIDE_SLOPE_ANGLE = None
        else:
          GLIDE_SLOPE_ANGLE = float(loc_se.group(37))/100
        STATION_DECLINATION = loc_se.group(38)
        if loc_se.group(39).strip() == "":
          GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = None
        else:
          GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD = loc_se.group(39)
        if loc_se.group(40).strip() == "":
          GLIDE_SLOPE_ELEV = None
        else:
          GLIDE_SLOPE_ELEV = loc_se.group(40)
        SUPPORTING_FACILITY_IDENT = loc_se.group(41)
        SUPPORTING_FACILITY_ICAO_CODE = loc_se.group(42)
        SUPPORTING_FACILITY_SECT_CODE = loc_se.group(43)
        SUPPORTING_FACILITY_SUBS_CODE = loc_se.group(44)
        LOC_RECD_NR = LOC_RECD_NR + 1
        CYCLE = loc_se.group(46)
        loc_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,ARPT_IDENT,ARPT_ICAO_CODE,SUBS_CODE,LOCALIZER_IDENT,ILS_CATEGORY,CONT_NR,LOCALIZER_FREQ,RUNWAY_IDENT,LOCALIZER_LAT,LOCALIZER_LON,LOCALIZER_BEARING,GLIDE_SLOPE_LAT,GLIDE_SLOPE_LON,LOCALIZER_POS,LOCALIZER_POS_REFERENCE,GLIDE_SLOPE_POS,LOCALIZER_WIDTH,GLIDE_SLOPE_ANGLE,STATION_DECLINATION,GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD,GLIDE_SLOPE_ELEV,SUPPORTING_FACILITY_IDENT,SUPPORTING_FACILITY_ICAO_CODE,SUPPORTING_FACILITY_SECT_CODE,SUPPORTING_FACILITY_SUBS_CODE,LOC_RECD_NR,CYCLE))
      if pn_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = pn_se.group(1)
        CUST_AREA = pn_se.group(2)
        SECT_CODE = pn_se.group(3)
        SUBS_CODE = pn_se.group(4)
        ARPT_IDENT = pn_se.group(5).strip()
        ARPT_ICAO_CODE = pn_se.group(6)
        NDB_IDENT = pn_se.group(7).strip()
        NDB_ICAO_CODE = pn_se.group(8)
        CONT_NR = pn_se.group(9)
        NDB_FREQ = float(pn_se.group(10))/10
        NDB_CLASS = pn_se.group(11)
        if pn_se.group(12) == "N":
          NDB_LAT = float(pn_se.group(13)) + float(pn_se.group(14))/60 + float(pn_se.group(15))/3600 + float(pn_se.group(16))/360000
        else:
          NDB_LAT = 0 - float(pn_se.group(13)) - float(pn_se.group(14))/60 - float(pn_se.group(15))/3600 - float(pn_se.group(16))/360000
        if pn_se.group(17) == "E":
          NDB_LON = float(pn_se.group(18)) + float(pn_se.group(19))/60 + float(pn_se.group(20))/3600 + float(pn_se.group(21))/360000
        else:
          NDB_LON = 0 - float(pn_se.group(18)) - float(pn_se.group(19))/60 - float(pn_se.group(20))/3600 - float(pn_se.group(21))/360000
        if pn_se.group(22) == "E":
          MAG_VAR = float(pn_se.group(23))/100
        else:
          MAG_VAR = 0 - float(pn_se.group(23))/100
        DATUM_CODE = pn_se.group(24)
        NDB_NAME = pn_se.group(25).replace('NDB','   ')
        NDB_RECD_NR = NDB_RECD_NR + 1
        CYCLE = pn_se.group(27)      
        ndb_fsl.append((RECD_TYPE,CUST_AREA,SECT_CODE,SUBS_CODE,ARPT_IDENT,ARPT_ICAO_CODE,NDB_IDENT,NDB_ICAO_CODE,CONT_NR,NDB_FREQ,NDB_CLASS,NDB_LAT,NDB_LON,MAG_VAR,DATUM_CODE,NDB_NAME,NDB_RECD_NR,CYCLE))
      if crz_se:
        if add_tailored:
          RECD_TYPE = "T"
        else:
          RECD_TYPE = crz_se.group(1)
        SECT_CODE = crz_se.group(2)
        SUBS_CODE = crz_se.group(3)
        CRUISE_TABLE_IDENT = crz_se.group(4)
        SEQUENCE_NUMBER = crz_se.group(5)
        COURSE_FROM = crz_se.group(6)
        COURSE_TO = crz_se.group(7)
        MAG_TRUE = crz_se.group(8)
        CRUISE_LEVEL_FROM_0 = crz_se.group(9)
        VERTICAL_SEPARATION_0 = crz_se.group(10)
        CRUISE_LEVEL_TO_0 = crz_se.group(11)
        CRUISE_LEVEL_FROM_1 = crz_se.group(12)
        VERTICAL_SEPARATION_1 = crz_se.group(13)
        CRUISE_LEVEL_TO_1 = crz_se.group(14)
        CRUISE_LEVEL_FROM_2 = crz_se.group(15)
        VERTICAL_SEPARATION_2 = crz_se.group(16)
        CRUISE_LEVEL_TO_2 = crz_se.group(17)
        CRUISE_LEVEL_FROM_3 = crz_se.group(18)
        VERTICAL_SEPARATION_3 = crz_se.group(19)
        CRUISE_LEVEL_TO_3 = crz_se.group(20)
        CRZ_RECD_NR = CRZ_RECD_NR + 1
        CYCLE = crz_se.group(22)
        crz_fsl.append((RECD_TYPE,SECT_CODE,SUBS_CODE,CRUISE_TABLE_IDENT,SEQUENCE_NUMBER,COURSE_FROM,COURSE_TO,MAG_TRUE,CRUISE_LEVEL_FROM_0,VERTICAL_SEPARATION_0,CRUISE_LEVEL_TO_0,CRUISE_LEVEL_FROM_1,VERTICAL_SEPARATION_1,CRUISE_LEVEL_TO_1,CRUISE_LEVEL_FROM_2,VERTICAL_SEPARATION_2,CRUISE_LEVEL_TO_2,CRUISE_LEVEL_FROM_3,VERTICAL_SEPARATION_3,CRUISE_LEVEL_TO_3,CRZ_RECD_NR,CYCLE))
    rom = sqlite3.connect('FSL.rom')
    rom_cur = rom.cursor()
    if new_database:
      rom_cur.execute('DROP TABLE IF EXISTS AIRPORT;')
      rom_cur.execute('DROP TABLE IF EXISTS AIRPORT_GATE;')
      rom_cur.execute('DROP TABLE IF EXISTS AIRPORT_LOCALIZER_AND_GLIDE_SLOPE;')
      rom_cur.execute('DROP TABLE IF EXISTS AIRPORT_PROCEDURE;')
      rom_cur.execute('DROP TABLE IF EXISTS COMPANY_ROUTE;')
      rom_cur.execute('DROP TABLE IF EXISTS CRUISING_TABLE;')
      rom_cur.execute('DROP TABLE IF EXISTS CYCLE_INFO;')
      rom_cur.execute('DROP TABLE IF EXISTS ENROUTE_AIRWAYS;')
      rom_cur.execute('DROP TABLE IF EXISTS GRID_MORA;')
      rom_cur.execute('DROP TABLE IF EXISTS HOLDING_PATTERN;')
      rom_cur.execute('DROP TABLE IF EXISTS NDB_NAVAID;')
      rom_cur.execute('DROP TABLE IF EXISTS RUNWAY;')
      rom_cur.execute('DROP TABLE IF EXISTS TACAN_ONLY_NAVAID;')
      rom_cur.execute('DROP TABLE IF EXISTS VHF_NAVAID;')
      rom_cur.execute('DROP TABLE IF EXISTS WAYPOINT;')
      rom_cur.execute('CREATE TABLE AIRPORT (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), IATA_IDENT varchar(3), CONT_NR varchar(1), SPEED_LIMIT_ALT real, LONGEST_RUNWAY varchar(3), IFR_CAPABILITY varchar(1), LONGEST_RUNWAY_SURFACE_CODE varchar(1), ARPT_LAT real, ARPT_LON real, MAG_VAR real, ARPT_ELEV real, SPEED_LIMIT real, RECOMMENDED_NAVAID varchar(4), RECOMMENDED_NAVAID_ICAO_CODE varchar(2), TRANSITIONS_ALT real, TRANSITION_LEVEL real, PUBLIC_MILITARY_IND varchar(1), TIME_ZONE varchar(3), DAYLIGHT_IND varchar(1), MAG_TRUE_IND varchar(1), DATUM_CODE varchar(3), ARPT_NAME varchar(30), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE AIRPORT_GATE (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), GATE_IDENT varchar(5), CONT_NR varchar(1), GATE_LAT real, GATE_LON real, NAME varchar(25), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE AIRPORT_LOCALIZER_AND_GLIDE_SLOPE (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), LOCALIZER_IDENT varchar(4), ILS_CATEGORY varchar(1), CONT_NR varchar(1), LOCALIZER_FREQ real, RUNWAY_IDENT varchar(5), LOCALIZER_LAT real, LOCALIZER_LON real, LOCALIZER_BEARING real, GLIDE_SLOPE_LAT real, GLIDE_SLOPE_LON real, LOCALIZER_POS varchar(4), LOCALIZER_POS_REFERENCE varchar(1), GLIDE_SLOPE_POS varchar(4), LOCALIZER_WIDTH real, GLIDE_SLOPE_ANGLE real, STATION_DECLINATION varchar(5), GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD real, GLIDE_SLOPE_ELEV real, SUPPORTING_FACILITY_IDENT varchar(4), SUPPORTING_FACILITY_ICAO_CODE varchar(2), SUPPORTING_FACILITY_SECT_CODE varchar(1), SUPPORTING_FACILITY_SUBS_CODE varchar(1), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE AIRPORT_PROCEDURE (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), PROC_IDENT varchar(6), ROUTE_TYPE varchar(1), TRANSITION_IDENT varchar(5), SEQ_NR real, FIX_IDENT varchar(5), FIX_ICAO_CODE varchar(2), FIX_SECT_CODE varchar(1), FIX_SUBS_CODE varchar(1), CONT_NR varchar(1), WAYPOINT_DESCR_CODE varchar(4), TURN_DIR varchar(1), RNP real, PATH_AND_TERMINATION varchar(2), TURN_DIR_VALID varchar(1), RECOMMENDED_NAVAID varchar(4), RECOMMENDED_NAVAID_ICAO_CODE varchar(2), ARC_RADIUS real, THETA real, RHO real, MAG_COURSE real, ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME varchar(4), RECOMMENDED_NAVAID_SECT_CODE varchar(1), RECOMMENDED_NAVAID_SUBS_CODE varchar(1), ALT_DESCR varchar(1), ATC_IND varchar(1), ALT_1 real, ALT_2 real, TRANSITION_ALT real, SPEED_LIMIT varchar(3), VERTICAL_ANGLE real, CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND varchar(5), MULTIPLE_CODE_OR_TAA_SECTOR_IDENT varchar(1), MULTIPLE_CODE_OR_TAA_SECTOR_ICAO_CODE varchar(2), MULTIPLE_CODE_OR_TAA_SECTOR_SECT_CODE varchar(1), MULTIPLE_CODE_OR_TAA_SECTOR_SUBS_CODE varchar(1), GNSS_FMS_INDICATION varchar(1), SPEED_LIMIT_DESCR varchar(1), APCH_ROUTE_QUALIFIER_1 varchar(1), APCH_ROUTE_QUALIFIER_2 varchar(1), VERTICAL_SCALE_FACTOR varchar(3), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE COMPANY_ROUTE (RECD_TYPE varchar(1), CUSTOMER varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), FROM_ARPT varchar(5), FROM_ARPT_ICAO_CODE varchar(2), FROM_ARPT_SECT_CODE varchar(1), FROM_ARPT_SUBS_CODE varchar(1), TO_ARPT varchar(5), TO_ARPT_ICAO_CODE varchar(2), TO_ARPT_SECT_CODE varchar(1), TO_ARPT_SUBS_CODE varchar(1), COMPANY_ROUTE_ID varchar(10), SEQUENCE_NO varchar(3), VIA varchar(3), SID_STAR_APP_AWY varchar(6), AREA_CODE varchar(3), TO_FIX varchar(6), TO_ICAO_CODE varchar(2), TO_SECT_CODE varchar(1), TO_SUBS_CODE varchar(1), RUNWAY_TRANS varchar(5), ENRT_TRANS varchar(5), CRUISE_ALT real, TERMINAL_ALTN_ARPT varchar(4), TERMINAL_ALTN_ARPT_ICAO_CODE varchar(2), ALTN_DISTANCE real, COST_INDEX varchar(3), ENROUTE_ALTN_ARPT varchar(4), SID_STAR_APP_AWY_ROUTE_TYPE varchar(1), S_S_A_ROUTE_TYPE_QUALIFIER_1 varchar(1), S_S_A_ROUTE_TYPE_QUALIFIER_2 varchar(1), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE CRUISING_TABLE (RECD_TYPE varchar(1), SECT_CODE varchar(1), SUBS_CODE varchar(1), CRUISE_TABLE_IDENT varchar(2), SEQUENCE_NUMBER varchar(1), COURSE_FROM varchar(4), COURSE_TO varchar(4), MAG_TRUE varchar(1), CRUISE_LEVEL_FROM_0 varchar(5), VERTICAL_SEPARATION_0 varchar(5), CRUISE_LEVEL_TO_0 varchar(5), CRUISE_LEVEL_FROM_1 varchar(5), VERTICAL_SEPARATION_1 varchar(5), CRUISE_LEVEL_TO_1 varchar(5), CRUISE_LEVEL_FROM_2 varchar(5), VERTICAL_SEPARATION_2 varchar(5), CRUISE_LEVEL_TO_2 varchar(5), CRUISE_LEVEL_FROM_3 varchar(5), VERTICAL_SEPARATION_3 varchar(5), CRUISE_LEVEL_TO_3 varchar(5), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE CYCLE_INFO(VARG, PARAMETER, VALUE);')
      rom_cur.execute('CREATE TABLE ENROUTE_AIRWAYS (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ROUTE_IDENT varchar(5), SEQ_NR real, FIX_IDENT varchar(5), FIX_ICAO_CODE varchar(2), FIX_SECT_CODE varchar(1), FIX_SUBS_CODE varchar(1), CONT_NR varchar(1), WAYPOINT_DESCR_CODE varchar(4), BOUNDARY_CODE varchar(1), ROUTE_TYPE varchar(1), LEVEL varchar(1), DIRECTION_RESTRICTION varchar(1), CRUISE_TABLE_IND varchar(2), EU_IND varchar(1), RECOMMENDED_NAVAID varchar(4), RECOMMENDED_NAVAID_ICAO_CODE varchar(2), RNP real, THETA real, RHO real, OUTBOUND_MAG_COURSE real, ROUTE_DISTANCE_FROM varchar(4), INBOUND_MAG_COURSE real, MINIMUM_ALT real, MINIMUM_ALT_1 real, MAXIMUM_ALT real, FIX_RADIUS_TRANSITION_IND varchar(3), VERTICAL_SCALE_FACTOR varchar(3), RVSM_MINIMUM_LEVEL real, VSF_RVSM_MAXIMUM_LEVEL real, ICAO_CODE_2 varchar(2), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE GRID_MORA (RECD_TYPE varchar(1), SECT_CODE varchar(1), SUBS_CODE varchar(1), MORA_LAT real, MORA_LON real, MORA_TYPE varchar(1), MORA_00 real, MORA_01 real, MORA_02 real, MORA_03 real, MORA_04 real, MORA_05 real, MORA_06 real, MORA_07 real, MORA_08 real, MORA_09 real, MORA_10 real, MORA_11 real, MORA_12 real, MORA_13 real, MORA_14 real, MORA_15 real, MORA_16 real, MORA_17 real, MORA_18 real, MORA_19 real, MORA_20 real, MORA_21 real, MORA_22 real, MORA_23 real, MORA_24 real, MORA_25 real, MORA_26 real, MORA_27 real, MORA_28 real, MORA_29 real, FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE HOLDING_PATTERN (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), REGION_CODE varchar(4), REGION_ICAO_CODE varchar(2), DUPLICATE_IDENT real, FIX_IDENT varchar(5), FIX_ICAO_CODE varchar(2), FIX_SECT_CODE varchar(1), FIX_SUBS_CODE varchar(1), CONT_NR varchar(1), INBOUND_HOLDING_COURSE real, TURN_DIR varchar(1), LEG_LENGTH real, LEG_TIME varchar(2), MINIMUM_ALT real, MAXIMUM_ALT real, HOLDING_SPEED real, RNP real, ARC_RADIUS real, VERTICAL_SCALE_FACTOR varchar(3), RVSM_MINIMUM_LEVEL real, RVSM_MAXIMUM_LEVEL real, NAME varchar(25), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE NDB_NAVAID (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), NDB_IDENT varchar(4), NDB_ICAO_CODE varchar(2), CONT_NR varchar(1), NDB_FREQ real, NDB_CLASS varchar(5), NDB_LAT real, NDB_LON real, MAG_VAR real, DATUM_CODE varchar(3), NDB_NAME varchar(30), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE RUNWAY (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), SUBS_CODE varchar(1), RUNWAY_IDENT varchar(5), CONT_NR varchar(1), RUNWAY_LENGTH real, RUNWAY_MAG_BEARING bearing, RUNWAY_LAT real, RUNWAY_LON real, RUNWAY_GRADIENT real, ELLIPSOID_HEIGHT real, LANDING_THLD_ELEV real, DISPLACED_THLD_DISTANCE real, THLD_CROSSING_HEIGHT real, RUNWAY_WIDTH real, TCH_VALUE_IND varchar(1), LOCALIZER_MLS_GLS_REF_PATH_IDENT varchar(4), LOCALIZER_MLS_GLS_CATEGORY_CLASS varchar(1), STOPWAY varchar(4), SECOND_LOCALIZER_MLS_GLS_REF_PATH_IDENT varchar(4), SECOND_LOCALIZER_MLS_GLS_CATEGORY_CLASS varchar(1), RUNWAY_DESCR varchar(22), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE TACAN_ONLY_NAVAID (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), VOR_IDENT varchar(4), CONT_NR varchar(1), VOR_FREQ real, NAVAID_CLASS varchar(5), TACAN_IDENT varchar(4), TACAN_LAT real, TACAN_LON real, THRU79_STATION_DECLINATION varchar(5), TACAN_ELEV real, FIGURE_OF_MERIT varchar(1), FREQ_PROTECTION varchar(3), DATUM_CODE varchar(3), TACAN_NAME varchar(30), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE VHF_NAVAID (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), ARPT_IDENT varchar(4), ARPT_ICAO_CODE varchar(2), VOR_IDENT varchar(4), VHF_ICAO_CODE varchar(2), CONT_NR varchar(1), VOR_FREQ real, NAVAID_CLASS varchar(5), VOR_LAT real, VOR_LON real, DME_IDENT varchar(4), DME_LAT real, DME_LON real, STATION_DECLINATION varchar(5), DME_ELEV real, FIGURE_OF_MERIT varchar(1), ILS_DME_BIAS real, FREQ_PROTECTION varchar(3), DATUM_CODE varchar(3), VOR_NAME varchar(25), ROUTE_INAPPROPRIATE_DME varchar(1), DME_OPERATIONAL_SERVICE_VOLUME varchar(1), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE TABLE WAYPOINT (RECD_TYPE varchar(1), CUST_AREA varchar(3), SECT_CODE varchar(1), SUBS_CODE varchar(1), REGION_CODE varchar(4), REGION_ICAO_CODE varchar(2), REGION_SUBS_CODE varchar(1), WAYPOINT_IDENT varchar(5), WAYPOINT_ICAO_CODE varchar(2), CONT_NR varchar(1), WAYPOINT_TYPE varchar(3), WAYPOINT_USAGE varchar(2), WAYPOINT_LAT real, WAYPOINT_LON real, DYNAMIC_MAG_VAR real, DATUM_CODE varchar(3), NAME_FORMAT_IND varchar(3), WAYPOINT_NAME_DESCR varchar(25), FILE_RECD_NR real, CYCLE real);')
      rom_cur.execute('CREATE INDEX IX_AIRPORT_ARPT_IDENT ON AIRPORT(ARPT_IDENT);')
      rom_cur.execute('CREATE INDEX IX_AIRPORT_ARPT_LAT_ARPT_LON ON AIRPORT(ARPT_LAT,ARPT_LON);')
      rom_cur.execute('CREATE INDEX IX_AIRPORT_GATE_ARPT_IDENT ON AIRPORT_GATE(ARPT_IDENT);')
      rom_cur.execute('CREATE INDEX IX_AIRPORT_GATE_RECD_TYPE ON AIRPORT_GATE(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_AIRPORT_LOCALIZER_AND_GLIDE_SLOPE_ARPT_IDENT ON AIRPORT_LOCALIZER_AND_GLIDE_SLOPE(ARPT_IDENT);')
      rom_cur.execute('CREATE INDEX IX_AIRPORT_LOCALIZER_AND_GLIDE_SLOPE_RECD_TYPE ON AIRPORT_LOCALIZER_AND_GLIDE_SLOPE(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_AIRPORT_PROCEDURE_ARPT_IDENT ON AIRPORT_PROCEDURE(ARPT_IDENT);')
      rom_cur.execute('CREATE INDEX IX_AIRPORT_PROCEDURE_RECD_TYPE ON AIRPORT_PROCEDURE(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_AIRPORT_RECD_TYPE ON AIRPORT(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_COMPANY_ROUTE_COMPANY_ROUTE_ID ON COMPANY_ROUTE(COMPANY_ROUTE_ID);')
      rom_cur.execute('CREATE INDEX IX_COMPANY_ROUTE_RECD_TYPE ON COMPANY_ROUTE(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_CRUISING_TABLE_RECD_TYPE ON CRUISING_TABLE(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_ENROUTE_AIRWAYS_RECD_TYPE ON ENROUTE_AIRWAYS(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_ENROUTE_AIRWAYS_ROUTE_IDENT_SEQ_NR_FIX_IDENT_FIX_ICAO_CODE_FIX_SECT_CODE_FIX_SUBS_CODE ON ENROUTE_AIRWAYS(ROUTE_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_CODE,FIX_SUBS_CODE);')
      rom_cur.execute('CREATE INDEX IX_GRID_MORA_RECD_TYPE ON GRID_MORA(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_HOLDING_PATTERN_RECD_TYPE ON HOLDING_PATTERN(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_NDB_FREQ ON NDB_NAVAID(NDB_FREQ);')
      rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_NDB_IDENT ON NDB_NAVAID(NDB_IDENT);')
      rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_NDB_LAT_NDB_LON ON NDB_NAVAID(NDB_LAT,NDB_LON);')
      rom_cur.execute('CREATE INDEX IX_NDB_NAVAID_RECD_TYPE ON NDB_NAVAID(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_RUNWAY_ARPT_IDENT ON RUNWAY(ARPT_IDENT);')
      rom_cur.execute('CREATE INDEX IX_RUNWAY_RECD_TYPE ON RUNWAY(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_TACAN_ONLY_NAVAID_RECD_TYPE ON TACAN_ONLY_NAVAID(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_DME_LAT_DME_LON ON VHF_NAVAID(DME_LAT,DME_LON);')
      rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_RECD_TYPE ON VHF_NAVAID(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_VOR_FREQ ON VHF_NAVAID(VOR_FREQ);')
      rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_VOR_IDENT ON VHF_NAVAID(VOR_IDENT);')
      rom_cur.execute('CREATE INDEX IX_VHF_NAVAID_VOR_LAT_VOR_LON ON VHF_NAVAID(VOR_LAT,VOR_LON);')
      rom_cur.execute('CREATE INDEX IX_WAYPOINT_RECD_TYPE ON WAYPOINT(RECD_TYPE);')
      rom_cur.execute('CREATE INDEX IX_WAYPOINT_WAYPOINT_IDENT ON WAYPOINT(WAYPOINT_IDENT);')
      rom_cur.execute('CREATE INDEX IX_WAYPOINT_WAYPOINT_LAT_WAYPOINT_LON ON WAYPOINT(WAYPOINT_LAT,WAYPOINT_LON);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_AIRPORT_FILE_RECD_NR ON AIRPORT(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_AIRPORT_GATE_FILE_RECD_NR ON AIRPORT_GATE(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_AIRPORT_LOCALIZER_AND_GLIDE_SLOPE_FILE_RECD_NR ON AIRPORT_LOCALIZER_AND_GLIDE_SLOPE(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_AIRPORT_PROCEDURE_FILE_RECD_NR ON AIRPORT_PROCEDURE(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_COMPANY_ROUTE_COMPANY_ROUTE_ID_SEQUENCE_NO ON COMPANY_ROUTE(COMPANY_ROUTE_ID,SEQUENCE_NO);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_COMPANY_ROUTE_FILE_RECD_NR ON COMPANY_ROUTE(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_CRUISING_TABLE_FILE_RECD_NR ON CRUISING_TABLE(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_ENROUTE_AIRWAYS_FILE_RECD_NR ON ENROUTE_AIRWAYS(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_GRID_MORA_FILE_RECD_NR ON GRID_MORA(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_GRID_MORA_MORA_LAT_MORA_LON_MORA_TYPE ON GRID_MORA(MORA_LAT,MORA_LON,MORA_TYPE);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_HOLDING_PATTERN_DUPLICATE_IDENT_FIX_IDENT_FIX_ICAO_CODE_FIX_SECT_CODE_FIX_SUBS_CODE_REGION_CODE_REGION_ICAO_CODE ON HOLDING_PATTERN(DUPLICATE_IDENT,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_CODE,FIX_SUBS_CODE,REGION_CODE,REGION_ICAO_CODE);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_HOLDING_PATTERN_FILE_RECD_NR ON HOLDING_PATTERN(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_NDB_NAVAID_FILE_RECD_NR ON NDB_NAVAID(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_RUNWAY_FILE_RECD_NR ON RUNWAY(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_TACAN_ONLY_NAVAID_FILE_RECD_NR ON TACAN_ONLY_NAVAID(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_VHF_NAVAID_FILE_RECD_NR ON VHF_NAVAID(FILE_RECD_NR);')
      rom_cur.execute('CREATE UNIQUE INDEX UX_WAYPOINT_FILE_RECD_NR ON WAYPOINT(FILE_RECD_NR);')
    if write_database:
      mora_fsl_tb = pandas.DataFrame(mora_fsl, columns=['RECD_TYPE','SECT_CODE','SUBS_CODE','MORA_LAT','MORA_LON','MORA_TYPE','MORA_00','MORA_01','MORA_02','MORA_03','MORA_04','MORA_05','MORA_06','MORA_07','MORA_08','MORA_09','MORA_10','MORA_11','MORA_12','MORA_13','MORA_14','MORA_15','MORA_16','MORA_17','MORA_18','MORA_19','MORA_20','MORA_21','MORA_22','MORA_23','MORA_24','MORA_25','MORA_26','MORA_27','MORA_28','MORA_29','FILE_RECD_NR','CYCLE'])
      vhf_fsl_tb = pandas.DataFrame(vhf_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','ARPT_IDENT','ARPT_ICAO_CODE','VOR_IDENT','VHF_ICAO_CODE','CONT_NR','VOR_FREQ','NAVAID_CLASS','VOR_LAT','VOR_LON','DME_IDENT','DME_LAT','DME_LON','STATION_DECLINATION','DME_ELEV','FIGURE_OF_MERIT','ILS_DME_BIAS','FREQ_PROTECTION','DATUM_CODE','VOR_NAME','ROUTE_INAPPROPRIATE_DME','DME_OPERATIONAL_SERVICE_VOLUME','FILE_RECD_NR','CYCLE'])
      ndb_fsl_tb = pandas.DataFrame(ndb_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','ARPT_IDENT','ARPT_ICAO_CODE','NDB_IDENT','NDB_ICAO_CODE','CONT_NR','NDB_FREQ','NDB_CLASS','NDB_LAT','NDB_LON','MAG_VAR','DATUM_CODE','NDB_NAME','FILE_RECD_NR','CYCLE'])
      awy_fsl_tb = pandas.DataFrame(awy_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','ROUTE_IDENT','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_SECT_CODE','FIX_SUBS_CODE','CONT_NR','WAYPOINT_DESCR_CODE','BOUNDARY_CODE','ROUTE_TYPE','LEVEL','DIRECTION_RESTRICTION','CRUISE_TABLE_IND','EU_IND','RECOMMENDED_NAVAID','RECOMMENDED_NAVAID_ICAO_CODE','RNP','THETA','RHO','OUTBOUND_MAG_COURSE','ROUTE_DISTANCE_FROM','INBOUND_MAG_COURSE','MINIMUM_ALT','MINIMUM_ALT_1','MAXIMUM_ALT','FIX_RADIUS_TRANSITION_IND','VERTICAL_SCALE_FACTOR','RVSM_MINIMUM_LEVEL','VSF_RVSM_MAXIMUM_LEVEL','ICAO_CODE_2','FILE_RECD_NR','CYCLE'])
      hold_fsl_tb = pandas.DataFrame(hold_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','REGION_CODE','REGION_ICAO_CODE','DUPLICATE_IDENT','FIX_IDENT','FIX_ICAO_CODE','FIX_SECT_CODE','FIX_SUBS_CODE','CONT_NR','INBOUND_HOLDING_COURSE','TURN_DIR','LEG_LENGTH','LEG_TIME','MINIMUM_ALT','MAXIMUM_ALT','HOLDING_SPEED','RNP','ARC_RADIUS','VERTICAL_SCALE_FACTOR','RVSM_MINIMUM_LEVEL','RVSM_MAXIMUM_LEVEL','NAME','FILE_RECD_NR','CYCLE'])
      arpt_fsl_tb = pandas.DataFrame(arpt_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','ARPT_IDENT','ARPT_ICAO_CODE','SUBS_CODE','IATA_IDENT','CONT_NR','SPEED_LIMIT_ALT','LONGEST_RUNWAY','IFR_CAPABILITY','LONGEST_RUNWAY_SURFACE_CODE','ARPT_LAT','ARPT_LON','MAG_VAR','ARPT_ELEV','SPEED_LIMIT','RECOMMENDED_NAVAID','RECOMMENDED_NAVAID_ICAO_CODE','TRANSITIONS_ALT','TRANSITION_LEVEL','PUBLIC_MILITARY_IND','TIME_ZONE','DAYLIGHT_IND','MAG_TRUE_IND','DATUM_CODE','ARPT_NAME','FILE_RECD_NR','CYCLE'])
      wpt_fsl_tb = pandas.DataFrame(wpt_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','SUBS_CODE','REGION_CODE','REGION_ICAO_CODE','REGION_SUBS_CODE','WAYPOINT_IDENT','WAYPOINT_ICAO_CODE','CONT_NR','WAYPOINT_TYPE','WAYPOINT_USAGE','WAYPOINT_LAT','WAYPOINT_LON','DYNAMIC_MAG_VAR','DATUM_CODE','NAME_FORMAT_IND','WAYPOINT_NAME_DESCR','FILE_RECD_NR','CYCLE'])
      proc_fsl_tb = pandas.DataFrame(proc_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','ARPT_IDENT','ARPT_ICAO_CODE','SUBS_CODE','PROC_IDENT','ROUTE_TYPE','TRANSITION_IDENT','SEQ_NR','FIX_IDENT','FIX_ICAO_CODE','FIX_SECT_CODE','FIX_SUBS_CODE','CONT_NR','WAYPOINT_DESCR_CODE','TURN_DIR','RNP','PATH_AND_TERMINATION','TURN_DIR_VALID','RECOMMENDED_NAVAID','RECOMMENDED_NAVAID_ICAO_CODE','ARC_RADIUS','THETA','RHO','MAG_COURSE','ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME','RECOMMENDED_NAVAID_SECT_CODE','RECOMMENDED_NAVAID_SUBS_CODE','ALT_DESCR','ATC_IND','ALT_1','ALT_2','TRANSITION_ALT','SPEED_LIMIT','VERTICAL_ANGLE','CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND','MULTIPLE_CODE_OR_TAA_SECTOR_IDENT','MULTIPLE_CODE_OR_TAA_SECTOR_ICAO_CODE','MULTIPLE_CODE_OR_TAA_SECTOR_SECT_CODE','MULTIPLE_CODE_OR_TAA_SECTOR_SUBS_CODE','GNSS_FMS_INDICATION','SPEED_LIMIT_DESCR','APCH_ROUTE_QUALIFIER_1','APCH_ROUTE_QUALIFIER_2','VERTICAL_SCALE_FACTOR','FILE_RECD_NR','CYCLE'])
      rwy_fsl_tb = pandas.DataFrame(rwy_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','ARPT_IDENT','ARPT_ICAO_CODE','SUBS_CODE','RUNWAY_IDENT','CONT_NR','RUNWAY_LENGTH','RUNWAY_MAG_BEARING','RUNWAY_LAT','RUNWAY_LON','RUNWAY_GRADIENT','ELLIPSOID_HEIGHT','LANDING_THLD_ELEV','DISPLACED_THLD_DISTANCE','THLD_CROSSING_HEIGHT','RUNWAY_WIDTH','TCH_VALUE_IND','LOCALIZER_MLS_GLS_REF_PATH_IDENT','LOCALIZER_MLS_GLS_CATEGORY_CLASS','STOPWAY','SECOND_LOCALIZER_MLS_GLS_REF_PATH_IDENT','SECOND_LOCALIZER_MLS_GLS_CATEGORY_CLASS','RUNWAY_DESCR','FILE_RECD_NR','CYCLE'])
      loc_fsl_tb = pandas.DataFrame(loc_fsl, columns=['RECD_TYPE','CUST_AREA','SECT_CODE','ARPT_IDENT','ARPT_ICAO_CODE','SUBS_CODE','LOCALIZER_IDENT','ILS_CATEGORY','CONT_NR','LOCALIZER_FREQ','RUNWAY_IDENT','LOCALIZER_LAT','LOCALIZER_LON','LOCALIZER_BEARING','GLIDE_SLOPE_LAT','GLIDE_SLOPE_LON','LOCALIZER_POS','LOCALIZER_POS_REFERENCE','GLIDE_SLOPE_POS','LOCALIZER_WIDTH','GLIDE_SLOPE_ANGLE','STATION_DECLINATION','GLIDE_SLOPE_HEIGHT_AT_LANDING_THLD','GLIDE_SLOPE_ELEV','SUPPORTING_FACILITY_IDENT','SUPPORTING_FACILITY_ICAO_CODE','SUPPORTING_FACILITY_SECT_CODE','SUPPORTING_FACILITY_SUBS_CODE','FILE_RECD_NR','CYCLE'])
      crz_fsl_tb = pandas.DataFrame(crz_fsl, columns=['RECD_TYPE','SECT_CODE','SUBS_CODE','CRUISE_TABLE_IDENT','SEQUENCE_NUMBER','COURSE_FROM','COURSE_TO','MAG_TRUE','CRUISE_LEVEL_FROM_0','VERTICAL_SEPARATION_0','CRUISE_LEVEL_TO_0','CRUISE_LEVEL_FROM_1','VERTICAL_SEPARATION_1','CRUISE_LEVEL_TO_1','CRUISE_LEVEL_FROM_2','VERTICAL_SEPARATION_2','CRUISE_LEVEL_TO_2','CRUISE_LEVEL_FROM_3','VERTICAL_SEPARATION_3','CRUISE_LEVEL_TO_3','FILE_RECD_NR','CYCLE'])
      mora_fsl_tb.to_sql('GRID_MORA', rom, if_exists='append', index=False)
      vhf_fsl_tb.to_sql('VHF_NAVAID', rom, if_exists='append', index=False)
      ndb_fsl_tb.to_sql('NDB_NAVAID', rom, if_exists='append', index=False)
      awy_fsl_tb.to_sql('ENROUTE_AIRWAYS', rom, if_exists='append', index=False)
      hold_fsl_tb.to_sql('HOLDING_PATTERN', rom, if_exists='append', index=False)
      arpt_fsl_tb.to_sql('AIRPORT', rom, if_exists='append', index=False)
      wpt_fsl_tb.to_sql('WAYPOINT', rom, if_exists='append', index=False)
      proc_fsl_tb.to_sql('AIRPORT_PROCEDURE', rom, if_exists='append', index=False)
      rwy_fsl_tb.to_sql('RUNWAY', rom, if_exists='append', index=False)
      loc_fsl_tb.to_sql('AIRPORT_LOCALIZER_AND_GLIDE_SLOPE', rom, if_exists='append', index=False)
      crz_fsl_tb.to_sql('CRUISING_TABLE', rom, if_exists='append', index=False) 
    if sinonav:
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_SUBS_CODE = " " WHERE FIX_IDENT = "DO" AND FIX_ICAO_CODE = "ZS" AND FIX_SECT_CODE = "D" AND FIX_SUBS_CODE = "B";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZS" WHERE FIX_IDENT = "DO" AND FIX_ICAO_CODE = "ZG" AND FIX_SECT_CODE = "D" AND FIX_SUBS_CODE = "B";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZG" WHERE FIX_IDENT = "AGTEL" AND FIX_ICAO_CODE = "ZJ";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZL" WHERE FIX_IDENT = "GOKEB" AND FIX_ICAO_CODE = "ZU";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZL" WHERE FIX_IDENT = "IGADA" AND FIX_ICAO_CODE = "ZH";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZS" WHERE FIX_IDENT = "OVTAN" AND FIX_ICAO_CODE = "ZG";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZS" WHERE FIX_IDENT = "SANKO" AND FIX_ICAO_CODE = "ZY";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZH" WHERE FIX_IDENT = "VADMO" AND FIX_ICAO_CODE = "ZS";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZH" WHERE FIX_IDENT = "WXI" AND FIX_ICAO_CODE = "ZB";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZY" WHERE FIX_IDENT = "HLD" AND FIX_ICAO_CODE = "ZB";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZY" WHERE FIX_IDENT = "UC" AND FIX_ICAO_CODE = "ZB" AND FIX_SECT_CODE = "D" AND FIX_SUBS_CODE = "B";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZY", FIX_SECT_CODE = "D", FIX_SUBS_CODE = "B" WHERE FIX_IDENT = "U" AND FIX_ICAO_CODE = "ZB" AND FIX_SECT_CODE = "P" AND FIX_SUBS_CODE = "N";')
      rom_cur.execute('UPDATE VHF_NAVAID SET VHF_ICAO_CODE = "ZB" WHERE ARPT_IDENT = "ZBMZ";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZS" WHERE FIX_IDENT = "ADGOL" AND FIX_ICAO_CODE = "ZH";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZL" WHERE FIX_IDENT = "ADMUX" AND FIX_ICAO_CODE = "ZW";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZY" WHERE FIX_IDENT = "ANOBI" AND FIX_ICAO_CODE = "ZB";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZG" WHERE FIX_IDENT = "DAPRO" AND FIX_ICAO_CODE = "ZH";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZU" WHERE FIX_IDENT = "IDEPO" AND FIX_ICAO_CODE = "ZG";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZY" WHERE FIX_IDENT = "IKARU" AND FIX_ICAO_CODE = "ZB";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZG" WHERE FIX_IDENT = "IKATA" AND FIX_ICAO_CODE = "ZS";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZS" WHERE FIX_IDENT = "IKUBA" AND FIX_ICAO_CODE = "ZH";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZW" WHERE FIX_IDENT = "LIKMI" AND FIX_ICAO_CODE = "ZL";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZL" WHERE FIX_IDENT = "MAGOD" AND FIX_ICAO_CODE = "ZW";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZP" WHERE FIX_IDENT = "MEMAG" AND FIX_ICAO_CODE = "ZU";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZY" WHERE FIX_IDENT = "MUDOT" AND FIX_ICAO_CODE = "ZB";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZS" WHERE FIX_IDENT = "NIXEP" AND FIX_ICAO_CODE = "ZY";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZW" WHERE FIX_IDENT = "OBDEG" AND FIX_ICAO_CODE = "ZL";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZS" WHERE FIX_IDENT = "OSVEN" AND FIX_ICAO_CODE = "ZY";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZB" WHERE FIX_IDENT = "OTBUL" AND FIX_ICAO_CODE = "ZL";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZY" WHERE FIX_IDENT = "OVTUP" AND FIX_ICAO_CODE = "ZB";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZB" WHERE FIX_IDENT = "PATVU" AND FIX_ICAO_CODE = "ZH";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZL" WHERE FIX_IDENT = "RUSDI" AND FIX_ICAO_CODE = "ZW";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZG" WHERE FIX_IDENT = "SAMAS" AND FIX_ICAO_CODE = "ZJ";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZB" WHERE FIX_IDENT = "SANID" AND FIX_ICAO_CODE = "ZL";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZG" WHERE FIX_IDENT = "SAPUT" AND FIX_ICAO_CODE = "ZS";')
      rom_cur.execute('UPDATE AIRPORT_PROCEDURE SET FIX_ICAO_CODE = "ZL" WHERE FIX_IDENT = "TUTNA" AND FIX_ICAO_CODE = "ZB";')
    if add_tailored:
      rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_SECT_CODE = "E" AND FIX_SUBS_CODE = "A" WHERE RECD_TYPE = "T" AND FIX_IDENT = "DO" AND FIX_ICAO_CODE = "ZS" AND FIX_SECT_CODE = "D" AND FIX_SUBS_CODE = "B";')
      rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_SUBS_CODE = "B" WHERE RECD_TYPE = "T" AND FIX_IDENT = "DO" AND FIX_ICAO_CODE = "ZS" AND FIX_SECT_CODE = "D" AND FIX_SUBS_CODE = " ";')
      rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZJ" WHERE RECD_TYPE = "T" AND FIX_IDENT = "AGTEL" AND FIX_ICAO_CODE = "ZG";')
      rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZU" WHERE RECD_TYPE = "T" AND FIX_IDENT = "GOKEB" AND FIX_ICAO_CODE = "ZL";')
      rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZH" WHERE RECD_TYPE = "T" AND FIX_IDENT = "IGADA" AND FIX_ICAO_CODE = "ZL";')
      rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZG" WHERE RECD_TYPE = "T" AND FIX_IDENT = "OVTAN" AND FIX_ICAO_CODE = "ZS";')
      rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZY" WHERE RECD_TYPE = "T" AND FIX_IDENT = "SANKO" AND FIX_ICAO_CODE = "ZS";')
      rom_cur.execute('UPDATE ENROUTE_AIRWAYS SET FIX_ICAO_CODE = "ZB" WHERE RECD_TYPE = "T" AND FIX_IDENT = "WXI" AND FIX_ICAO_CODE = "ZH";')
      rom_cur.execute('UPDATE NDB_NAVAID SET NDB_ICAO_CODE = "ZG" WHERE RECD_TYPE = "T" AND NDB_IDENT = "DO" AND NDB_ICAO_CODE = "ZS";')
    rom_cur.close()
    rom.commit()
    rom.close()
    
  # JeeHell (Unfinished)
  if JeeHell:
    mora_jh = []
    vhf_jh = []
    ndb_jh = []
    hold_jh = []
    awy_jh = []
    arpt_jh = []
    wpt_jh = []
    proc_jh = []
    rwy_jh = []
    loc_jh = []
    crz_jh = []
    for line in ARINC424:
      mora_se = re.search(as_grid, line)
      vhf_se = re.search(vor, line)
      db_se = re.search(db, line)
      ea_se = re.search(ea, line)
      hold_se = re.search(ep, line)
      #awy_se = re.search(er, line)
      arpt_se = re.search(pa, line)
      pc_se = re.search(pc, line)
      proc_se = re.search(pdef, line)
      rwy_se = re.search(pg, line)
      loc_se = re.search(pi, line)
      pn_se = re.search(pn, line)
      crz_se = re.search(tc, line)
      if vhf_se:
        SECT_SUBS = vhf_se.group(3) + vhf_se.group(4)
        VOR_IDENT = vhf_se.group(7).strip()
        VHF_ICAO_CODE = vhf_se.group(8)
        VOR_FREQ = float(vhf_se.group(10))/100
        NAVAID_CLASS = vhf_se.group(11)
        if vhf_se.group(12).strip() == "":
          VOR_LAT = None
        elif vhf_se.group(12) == "N":
          VOR_LAT = round(float(vhf_se.group(13)) + float(vhf_se.group(14))/60 + float(vhf_se.group(15))/3600 + float(vhf_se.group(16))/360000, 8)
        else:
          VOR_LAT = round(0 - float(vhf_se.group(13)) - float(vhf_se.group(14))/60 - float(vhf_se.group(15))/3600 - float(vhf_se.group(16))/360000, 8)
        if vhf_se.group(17).strip() == "":
          VOR_LON = None
        elif vhf_se.group(17) == "E":
          VOR_LON = round(float(vhf_se.group(18)) + float(vhf_se.group(19))/60 + float(vhf_se.group(20))/3600 + float(vhf_se.group(21))/360000, 8)
        else:
          VOR_LON = round(0 - float(vhf_se.group(18)) - float(vhf_se.group(19))/60 - float(vhf_se.group(20))/3600 - float(vhf_se.group(21))/360000, 8)
        if vhf_se.group(23).strip() == "":
          DME_LAT = None
        elif vhf_se.group(23) == "N":
          DME_LAT = round(float(vhf_se.group(24)) + float(vhf_se.group(25))/60 + float(vhf_se.group(26))/3600 + float(vhf_se.group(27))/360000, 8)
        else:
          DME_LAT = round(0 - float(vhf_se.group(24)) - float(vhf_se.group(25))/60 - float(vhf_se.group(26))/3600 - float(vhf_se.group(27))/360000, 8)
        if vhf_se.group(28).strip() == "":
          DME_LON = None
        elif vhf_se.group(28) == "E":
          DME_LON = round(float(vhf_se.group(29)) + float(vhf_se.group(30))/60 + float(vhf_se.group(31))/3600 + float(vhf_se.group(32))/360000, 8)
        else:
          DME_LON = round(0 - float(vhf_se.group(29)) - float(vhf_se.group(30))/60 - float(vhf_se.group(31))/3600 - float(vhf_se.group(32))/360000, 8)
        if vhf_se.group(33).strip() == "":
          STATION_DECLINATION = ""
        else:
          STATION_DECLINATION = vhf_se.group(33).strip(string.digits) + str(float(vhf_se.group(33).strip('EW'))/10)
        if vhf_se.group(34).strip() == "":
          DME_ELEV = None
        else:
          DME_ELEV = vhf_se.group(34).lstrip('0')
        VOR_NAME = vhf_se.group(39).replace('VOR/DME','').rstrip()
        vhf_jh.append((SECT_SUBS,VOR_IDENT,VHF_ICAO_CODE,VOR_FREQ,NAVAID_CLASS,VOR_LAT,VOR_LON,DME_LAT,DME_LON,STATION_DECLINATION,DME_ELEV,"0.0",VOR_NAME,""))
      if db_se:
        SECT_SUBS = db_se.group(3) + db_se.group(4)
        NDB_IDENT = db_se.group(7).strip()
        NDB_ICAO_CODE = db_se.group(8)
        NDB_FREQ = float(db_se.group(10))/10
        NDB_CLASS = db_se.group(11)
        if db_se.group(12) == "N":
          NDB_LAT = round(float(db_se.group(13)) + float(db_se.group(14))/60 + float(db_se.group(15))/3600 + float(db_se.group(16))/360000, 8)
        else:
          NDB_LAT = round(0 - float(db_se.group(13)) - float(db_se.group(14))/60 - float(db_se.group(15))/3600 - float(db_se.group(16))/360000, 8)
        if db_se.group(17) == "E":
          NDB_LON = round(float(db_se.group(18)) + float(db_se.group(19))/60 + float(db_se.group(20))/3600 + float(db_se.group(21))/360000, 8)
        else:
          NDB_LON = round(0 - float(db_se.group(18)) - float(db_se.group(19))/60 - float(db_se.group(20))/3600 - float(db_se.group(21))/360000, 8)
        NDB_NAME = db_se.group(25).replace('NDB','').rstrip()
        ndb_jh.append((SECT_SUBS,NDB_IDENT,NDB_ICAO_CODE,NDB_FREQ,NDB_CLASS,NDB_LAT,NDB_LON,NDB_NAME,""))
      if ea_se:
        SECT_SUBS = ea_se.group(3) + ea_se.group(4)
        REGION_CODE = ea_se.group(5)
        WAYPOINT_IDENT = ea_se.group(8).strip()
        WAYPOINT_ICAO_CODE = ea_se.group(9)
        WAYPOINT_TYPE = ea_se.group(11)
        if ea_se.group(13) == "N":
          WAYPOINT_LAT = round(float(ea_se.group(14)) + float(ea_se.group(15))/60 + float(ea_se.group(16))/3600 + float(ea_se.group(17))/360000, 8)
        else:
          WAYPOINT_LAT = round(0 - float(ea_se.group(14)) - float(ea_se.group(15))/60 - float(ea_se.group(16))/3600 - float(ea_se.group(17))/360000, 8)
        if ea_se.group(18) == "E":
          WAYPOINT_LON = round(float(ea_se.group(19)) + float(ea_se.group(20))/60 + float(ea_se.group(21))/3600 + float(ea_se.group(22))/360000, 8)
        else:
          WAYPOINT_LON = round(0 - float(ea_se.group(19)) - float(ea_se.group(20))/60 - float(ea_se.group(21))/3600 - float(ea_se.group(22))/360000, 8)
        WAYPOINT_NAME_DESCR = ea_se.group(26).rstrip()
        wpt_jh.append((SECT_SUBS,WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WAYPOINT_TYPE,WAYPOINT_LAT,WAYPOINT_LON,REGION_CODE,WAYPOINT_NAME_DESCR,""))
      '''if awy_se:
        SECT_SUBS = awy_se.group(3) + awy_se.group(4)
        ROUTE_IDENT = awy_se.group(5).strip()
        SEQ_NR = awy_se.group(6)
        FIX_IDENT = awy_se.group(7).strip()
        FIX_ICAO_CODE = awy_se.group(8)
        FIX_SECT_SUBS = awy_se.group(9) + awy_se.group(10)
        WAYPOINT_DESCR_CODE = awy_se.group(12)
        LEVEL = awy_se.group(15).strip()
        DIRECTION_RESTRICTION = awy_se.group(16).strip()
        if awy_se.group(27).strip() == "":
          MINIMUM_ALT = None
        elif awy_se.group(27).rstrip(string.digits) == "FL":
          MINIMUM_ALT = awy_se.group(27).lstrip(string.ascii_uppercase) + "00"
        else:
          MINIMUM_ALT = awy_se.group(27)
        if awy_se.group(28).strip() == "":
          MINIMUM_ALT_1 = None
        elif awy_se.group(28).rstrip(string.digits) == "FL":
          MINIMUM_ALT_1 = awy_se.group(28).lstrip(string.ascii_uppercase) + "00"
        else:
          MINIMUM_ALT_1 = awy_se.group(28)
        if awy_se.group(29).strip() == "":
          MAXIMUM_ALT = None
        elif awy_se.group(29).rstrip(string.digits) == "FL":
          MAXIMUM_ALT = awy_se.group(29).lstrip(string.ascii_uppercase) + "00"
        else:
          MAXIMUM_ALT = awy_se.group(29)
        awy_jh.append((SECT_SUBS,ROUTE_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_SUBS,WAYPOINT_DESCR_CODE,'','','',WAYPOINT_LAT,WAYPOINT_LON,LEVEL,DIRECTION_RESTRICTION,MINIMUM_ALT,MINIMUM_ALT_1,MAXIMUM_ALT,'',''))'''
      if arpt_se:
        SECT_SUBS = arpt_se.group(3) + arpt_se.group(6)
        ARPT_IDENT = arpt_se.group(4)
        ARPT_ICAO_CODE = arpt_se.group(5)
        if arpt_se.group(9).strip() == "":
          SPEED_LIMIT_ALT = None
        else:
          SPEED_LIMIT_ALT = arpt_se.group(9).lstrip('0')
        if arpt_se.group(13) == "N":
          ARPT_LAT = round(float(arpt_se.group(14)) + float(arpt_se.group(15))/60 + float(arpt_se.group(16))/3600 + float(arpt_se.group(17))/360000, 8)
        else:
          ARPT_LAT = round(0 - float(arpt_se.group(14)) - float(arpt_se.group(15))/60 - float(arpt_se.group(16))/3600 - float(arpt_se.group(17))/360000, 8)
        if arpt_se.group(18) == "E":
          ARPT_LON = round(float(arpt_se.group(19)) + float(arpt_se.group(20))/60 + float(arpt_se.group(21))/3600 + float(arpt_se.group(22))/360000, 8)
        else:
          ARPT_LON = round(0 - float(arpt_se.group(19)) - float(arpt_se.group(20))/60 - float(arpt_se.group(21))/3600 - float(arpt_se.group(22))/360000, 8)
        ARPT_ELEV = arpt_se.group(25).lstrip('0')
        if arpt_se.group(26).strip() == "":
          SPEED_LIMIT = None
        else:
          SPEED_LIMIT = arpt_se.group(26)
        if arpt_se.group(29).strip() == "":
          TRANSITIONS_ALT = None
        else:
          TRANSITIONS_ALT = arpt_se.group(29).lstrip('0')
        ARPT_NAME = arpt_se.group(36).rstrip()
        arpt_jh.append((SECT_SUBS,ARPT_IDENT,ARPT_ICAO_CODE,ARPT_LAT,ARPT_LON,ARPT_ELEV,SPEED_LIMIT,SPEED_LIMIT_ALT,TRANSITIONS_ALT,ARPT_NAME,''))
      if pc_se:
        SECT_SUBS = pc_se.group(3) + pc_se.group(7)
        REGION_CODE = pc_se.group(5)
        WAYPOINT_IDENT = pc_se.group(8).strip()
        WAYPOINT_ICAO_CODE = pc_se.group(9)
        WAYPOINT_TYPE = pc_se.group(11)
        if pc_se.group(13) == "N":
          WAYPOINT_LAT = round(float(pc_se.group(14)) + float(pc_se.group(15))/60 + float(pc_se.group(16))/3600 + float(pc_se.group(17))/360000, 8)
        else:
          WAYPOINT_LAT = round(0 - float(pc_se.group(14)) - float(pc_se.group(15))/60 - float(pc_se.group(16))/3600 - float(pc_se.group(17))/360000, 8)
        if pc_se.group(18) == "E":
          WAYPOINT_LON = round(float(pc_se.group(19)) + float(pc_se.group(20))/60 + float(pc_se.group(21))/3600 + float(pc_se.group(22))/360000, 8)
        else:
          WAYPOINT_LON = round(0 - float(pc_se.group(19)) - float(pc_se.group(20))/60 - float(pc_se.group(21))/3600 - float(pc_se.group(22))/360000, 8)
        WAYPOINT_NAME_DESCR = pc_se.group(26).rstrip()
        wpt_jh.append((SECT_SUBS,WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WAYPOINT_TYPE,WAYPOINT_LAT,WAYPOINT_LON,REGION_CODE,WAYPOINT_NAME_DESCR,""))
      if proc_se:
        SECT_SUBS = proc_se.group(3) + proc_se.group(6)
        ARPT_IDENT = proc_se.group(4)
        ARPT_ICAO_CODE = proc_se.group(5)
        PROC_IDENT = proc_se.group(7).strip()
        ROUTE_TYPE = proc_se.group(8)
        TRANSITION_IDENT = proc_se.group(9).strip()
        SEQ_NR = proc_se.group(10)
        FIX_IDENT = proc_se.group(11).strip()
        FIX_ICAO_CODE = proc_se.group(12)
        FIX_SECT_SUBS = proc_se.group(13) + proc_se.group(14)
        CONT_NR = proc_se.group(15)
        WAYPOINT_DESCR_CODE = proc_se.group(16)
        TURN_DIR = proc_se.group(17)
        if proc_se.group(18).strip() == "":
          RNP = None
        else:
          RNP = float(proc_se.group(18))//10 / 10**(float(proc_se.group(18))%10)
        PATH_AND_TERMINATION = proc_se.group(19)
        TURN_DIR_VALID = proc_se.group(20)
        RECOMMENDED_NAVAID = proc_se.group(21)
        RECOMMENDED_NAVAID_ICAO_CODE = proc_se.group(22)
        if proc_se.group(23).strip() == "":
          ARC_RADIUS = None
        else:
          ARC_RADIUS = round(float(proc_se.group(23))/1000, 4)
        if proc_se.group(24).strip() == "":
          THETA = None
        else:
          THETA = proc_se.group(24)
        if proc_se.group(25).strip() == "":
          RHO = None
        else:
          RHO = proc_se.group(25)
        if proc_se.group(26).strip() == "":
          MAG_COURSE = None
        elif proc_se.group(26).lstrip(string.digits) == "T":
          MAG_COURSE = proc_se.group(26).rstrip('T')
        else:
          MAG_COURSE = float(proc_se.group(26))//10
        if proc_se.group(27).strip() == "":
          ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME = None
        elif proc_se.group(27).rstrip(string.digits) == "T":
          ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME = float(proc_se.group(27).lstrip('T'))/10
        else:
          ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME = float(proc_se.group(27))/10
        RECOMMENDED_NAVAID_SECT_SUBS = proc_se.group(28) + proc_se.group(29).strip()
        if proc_se.group(32).strip() == "":
          ALT_DESCR = None
        elif proc_se.group(30).strip() == "":
          ALT_DESCR = '@'
        else:
          ALT_DESCR = proc_se.group(30)
        if proc_se.group(32).strip() == "":
          ALT_1 = None
        elif proc_se.group(32).rstrip(string.digits) == "FL":
          ALT_1 = proc_se.group(32).lstrip(string.ascii_uppercase) + "00"
        else:
          ALT_1 = proc_se.group(32).lstrip('0')
        if proc_se.group(33).strip() == "":
          ALT_2 = None
        elif proc_se.group(33).rstrip(string.digits) == "FL":
          ALT_2 = proc_se.group(33).lstrip(string.ascii_uppercase) + "00"
        else:
          ALT_2 = proc_se.group(33).lstrip('0')
        if proc_se.group(34).strip() == "":
          TRANSITION_ALT = None
        else:
          TRANSITION_ALT = proc_se.group(34).lstrip('0')
        SPEED_LIMIT = proc_se.group(35)
        if proc_se.group(36).strip() == "":
          VERTICAL_ANGLE = None
        else:
          VERTICAL_ANGLE = round(float(proc_se.group(36))/100, 2)
        CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND = proc_se.group(37)
        MULTIPLE_CODE_OR_TAA_SECTOR_IDENT = proc_se.group(38).strip()
        MULTIPLE_CODE_OR_TAA_SECTOR_SECT_SUBS = proc_se.group(40) + proc_se.group(41).strip()
        SPEED_LIMIT_DESCR = proc_se.group(43)
        APCH_ROUTE_QUALIFIER_1 = proc_se.group(44)
        APCH_ROUTE_QUALIFIER_2 = proc_se.group(45)
        proc_jh.append((SECT_SUBS,ARPT_IDENT,ARPT_ICAO_CODE,PROC_IDENT,ROUTE_TYPE,TRANSITION_IDENT,SEQ_NR,FIX_IDENT,FIX_ICAO_CODE,FIX_SECT_SUBS,WAYPOINT_DESCR_CODE,TURN_DIR,TURN_DIR_VALID,RNP,PATH_AND_TERMINATION,RECOMMENDED_NAVAID,RECOMMENDED_NAVAID_ICAO_CODE,RECOMMENDED_NAVAID_SECT_SUBS,ARC_RADIUS,THETA,RHO,MAG_COURSE,ROUTE_DISTANCE_HOLDING_DISTANCE_OR_TIME,ALT_DESCR,ALT_1,ALT_2,SPEED_LIMIT,SPEED_LIMIT_DESCR,VERTICAL_ANGLE,CENTER_FIX_OR_TAA_PROCEDURE_TURN_IND,TRANSITION_ALT,MULTIPLE_CODE_OR_TAA_SECTOR_IDENT,MULTIPLE_CODE_OR_TAA_SECTOR_SECT_SUBS,CONT_NR,APCH_ROUTE_QUALIFIER_1,APCH_ROUTE_QUALIFIER_2,''))
      if rwy_se:
        SECT_SUBS = rwy_se.group(3) + rwy_se.group(6)
        ARPT_IDENT = rwy_se.group(4)
        ARPT_ICAO_CODE = rwy_se.group(5)
        RUNWAY_IDENT = rwy_se.group(7).strip()
        RUNWAY_LENGTH = rwy_se.group(9)
        if rwy_se.group(10).lstrip(string.digits) == "T":
          RUNWAY_MAG_BEARING = rwy_se.group(10).rstrip('T')
        else:
          RUNWAY_MAG_BEARING = float(rwy_se.group(10))/10
        if rwy_se.group(11) == "N":
          RUNWAY_LAT = round(float(rwy_se.group(12)) + float(rwy_se.group(13))/60 + float(rwy_se.group(14))/3600 + float(rwy_se.group(15))/360000, 8)
        else:
          RUNWAY_LAT = round(0 - float(rwy_se.group(12)) - float(rwy_se.group(13))/60 - float(rwy_se.group(14))/3600 - float(rwy_se.group(15))/360000, 8)
        if rwy_se.group(16) == "E":
          RUNWAY_LON = round(float(rwy_se.group(17)) + float(rwy_se.group(18))/60 + float(rwy_se.group(19))/3600 + float(rwy_se.group(20))/360000, 8)
        else:
          RUNWAY_LON = round(0 - float(rwy_se.group(17)) - float(rwy_se.group(18))/60 - float(rwy_se.group(19))/3600 - float(rwy_se.group(20))/360000, 8)
        LANDING_THLD_ELEV = rwy_se.group(24).lstrip('0')
        if rwy_se.group(25).strip() == "":
          DISPLACED_THLD_DISTANCE = "0"
        else:
          DISPLACED_THLD_DISTANCE = rwy_se.group(25).lstrip('0')
        RUNWAY_WIDTH = rwy_se.group(27)
        LOCALIZER_MLS_GLS_REF_PATH_IDENT = rwy_se.group(29).strip()
        LOCALIZER_MLS_GLS_CATEGORY_CLASS = rwy_se.group(30)
        rwy_jh.append((SECT_SUBS,ARPT_IDENT,ARPT_ICAO_CODE,RUNWAY_IDENT,RUNWAY_LENGTH,DISPLACED_THLD_DISTANCE,RUNWAY_WIDTH,RUNWAY_LAT,RUNWAY_LON,LANDING_THLD_ELEV,RUNWAY_MAG_BEARING,LOCALIZER_MLS_GLS_REF_PATH_IDENT,LOCALIZER_MLS_GLS_CATEGORY_CLASS,''))
      if pn_se:
        SECT_SUBS = db_se.group(3) + db_se.group(4)
        NDB_IDENT = db_se.group(7).strip()
        NDB_ICAO_CODE = db_se.group(8)
        NDB_FREQ = float(db_se.group(10))/10
        NDB_CLASS = db_se.group(11)
        if db_se.group(12) == "N":
          NDB_LAT = round(float(db_se.group(13)) + float(db_se.group(14))/60 + float(db_se.group(15))/3600 + float(db_se.group(16))/360000, 8)
        else:
          NDB_LAT = round(0 - float(db_se.group(13)) - float(db_se.group(14))/60 - float(db_se.group(15))/3600 - float(db_se.group(16))/360000, 8)
        if db_se.group(17) == "E":
          NDB_LON = round(float(db_se.group(18)) + float(db_se.group(19))/60 + float(db_se.group(20))/3600 + float(db_se.group(21))/360000, 8)
        else:
          NDB_LON = round(0 - float(db_se.group(18)) - float(db_se.group(19))/60 - float(db_se.group(20))/3600 - float(db_se.group(21))/360000, 8)
        NDB_NAME = db_se.group(25).replace('NDB','').rstrip()
        ndb_jh.append((SECT_SUBS,NDB_IDENT,NDB_ICAO_CODE,NDB_FREQ,NDB_CLASS,NDB_LAT,NDB_LON,NDB_NAME,""))
    if write_database:
      mora_jh_tb = pandas.DataFrame(mora_jh)
      vhf_jh_tb = pandas.DataFrame(vhf_jh)
      ndb_jh_tb = pandas.DataFrame(ndb_jh)
      #awy_jh_tb = pandas.DataFrame(awy_jh)
      hold_jh_tb = pandas.DataFrame(hold_jh)
      arpt_jh_tb = pandas.DataFrame(arpt_jh)
      wpt_jh_tb = pandas.DataFrame(wpt_jh)
      proc_jh_tb = pandas.DataFrame(proc_jh)
      rwy_jh_tb = pandas.DataFrame(rwy_jh)
      loc_jh_tb = pandas.DataFrame(loc_jh)
      crz_jh_tb = pandas.DataFrame(crz_jh)
      vhf_jh_tb.to_csv(path_or_buf='JeeHell-VOR.txt', sep='|', header=False, index=False)
      ndb_jh_tb.to_csv(path_or_buf='JeeHell-NDB.txt', sep='|', header=False, index=False)
      #awy_jh_tb.to_csv(path_or_buf='JeeHell-AWY.txt', sep='|', header=False, index=False)
      wpt_jh_tb.to_csv(path_or_buf='JeeHell-WPT.txt', sep='|', header=False, index=False)
      arpt_jh_tb.to_csv(path_or_buf='JeeHell-ARPT.txt', sep='|', header=False, index=False)
      proc_jh_tb.to_csv(path_or_buf='JeeHell-PROC.txt', sep='|', header=False, index=False)
      rwy_jh_tb.to_csv(path_or_buf='JeeHell-RWY.txt', sep='|', header=False, index=False)

  # MJC (Unfinished)
  if MJC:
    AirportLookup_mjc = []
    Airports_mjc = []
    AirwayLegs_mjc = []
    Airways_mjc = []
    ILSes_mjc = []
    NavaidLookup_mjc = []
    Navaids_mjc = []
    Runways_mjc = []
    TerminalLegs_mjc = []
    TerminalLegsEx_mjc = []
    Terminals_mjc = []
    WaypointLookup_mjc = []
    Waypoints_mjc = []
    AirportID = 15000
    NavaidID = 290000
    WaypointID = 300000
    RunwayID = 40000
    AirwayID = 10000
    TerminalID = 100000
    TerminalLegsID = 800000
    for line in ARINC424:
      vhf_se = re.search(vor, line)
      db_se = re.search(db, line)
      ea_se = re.search(ea, line)
      #awy_se = re.search(er, line)
      arpt_se = re.search(pa, line)
      pc_se = re.search(pc, line)
      proc_se = re.search(pdef, line)
      rwy_se = re.search(pg, line)
      loc_se = re.search(pi, line)
      pn_se = re.search(pn, line)
      if vhf_se:
        VOR_IDENT = vhf_se.group(7).strip()
        VHF_ICAO_CODE = vhf_se.group(8)
        VOR_FREQ = int('0x' + str(int(vhf_se.group(10))*100) , 16)
        if vhf_se.group(11)[0:1] == "V":
          if vhf_se.group(11)[1:2] == " ":
            NAVAID_TYPE = 1
          elif vhf_se.group(11)[1:2] == "T":
            NAVAID_TYPE = 2
          elif vhf_se.group(11)[1:2] == "M":
            NAVAID_TYPE = 2
          elif vhf_se.group(11)[1:2] == "D":
            NAVAID_TYPE = 4
          else:
            NAVAID_TYPE = 1
        else:
          if vhf_se.group(11)[1:2] == "T":
            NAVAID_TYPE = 3
          elif vhf_se.group(11)[1:2] == "M":
            NAVAID_TYPE = 3
          elif vhf_se.group(11)[1:2] == "I":
            NAVAID_TYPE = 8          
          elif vhf_se.group(11)[1:2] == "D":
            NAVAID_TYPE = 9
          else:
            NAVAID_TYPE = 9
        if vhf_se.group(11)[2:3] == "U":
          NAVAID_CLASS = "B"
        else :
          NAVAID_CLASS = vhf_se.group(11)[2:3]
        if vhf_se.group(12).strip() == "":
          if vhf_se.group(23) == "N":
            VOR_LAT = float(vhf_se.group(24)) + float(vhf_se.group(25))/60 + float(vhf_se.group(26))/3600 + float(vhf_se.group(27))/360000
          else:
            VOR_LAT = 0 - float(vhf_se.group(24)) - float(vhf_se.group(25))/60 - float(vhf_se.group(26))/3600 - float(vhf_se.group(27))/360000
        elif vhf_se.group(12) == "N":
          VOR_LAT = float(vhf_se.group(13)) + float(vhf_se.group(14))/60 + float(vhf_se.group(15))/3600 + float(vhf_se.group(16))/360000
        else:
          VOR_LAT = 0 - float(vhf_se.group(13)) - float(vhf_se.group(14))/60 - float(vhf_se.group(15))/3600 - float(vhf_se.group(16))/360000
        if vhf_se.group(17).strip() == "":
          if vhf_se.group(28) == "E":
            VOR_LON = float(vhf_se.group(29)) + float(vhf_se.group(30))/60 + float(vhf_se.group(31))/3600 + float(vhf_se.group(32))/360000
          else:
            VOR_LON = 0 - float(vhf_se.group(29)) - float(vhf_se.group(30))/60 - float(vhf_se.group(31))/3600 - float(vhf_se.group(32))/360000
        elif vhf_se.group(17) == "E":
          VOR_LON = float(vhf_se.group(18)) + float(vhf_se.group(19))/60 + float(vhf_se.group(20))/3600 + float(vhf_se.group(21))/360000
        else:
          VOR_LON = 0 - float(vhf_se.group(18)) - float(vhf_se.group(19))/60 - float(vhf_se.group(20))/3600 - float(vhf_se.group(21))/360000
        if vhf_se.group(33).strip() == "":
          STATION_DECLINATION = 0
        elif vhf_se.group(33).strip(string.digits) == "E":
          STATION_DECLINATION = float(vhf_se.group(33).strip('EW'))/10
        else:
          STATION_DECLINATION = 0 - float(vhf_se.group(33).strip('EW'))/10
        if vhf_se.group(34).strip() == "":
          DME_ELEV = 0
        else:
          DME_ELEV = vhf_se.group(34)
        VOR_NAME = vhf_se.group(39).replace('VOR/DME','').rstrip()
        NavaidID = NavaidID + 1
        Navaids_mjc.append((NavaidID,VOR_IDENT,NAVAID_TYPE,VOR_NAME,VOR_FREQ,None,NAVAID_CLASS,VOR_LAT,VOR_LON,DME_ELEV,STATION_DECLINATION))
        NavaidLookup_mjc.append((VOR_IDENT,NAVAID_TYPE,VHF_ICAO_CODE,1,NavaidID))
        Waypoints_mjc.append((NavaidID,VOR_IDENT,1,VOR_NAME,VOR_LAT,VOR_LON,int(NavaidID)))
        WaypointLookup_mjc.append((VOR_IDENT,VHF_ICAO_CODE,NavaidID))
      if db_se:
        NDB_IDENT = db_se.group(7).strip()
        NDB_ICAO_CODE = db_se.group(8)
        NDB_FREQ = int('0x' + str(int(db_se.group(10))*1000) , 16)
        NAVAID_TYPE = 5
        if db_se.group(11)[2:3] == "L":
          NAVAID_CLASS = "T"
        elif db_se.group(11)[2:3] == "M":
          NAVAID_CLASS = "L"
        else :
          NAVAID_CLASS = "H"
        if db_se.group(12) == "N":
          NDB_LAT = float(db_se.group(13)) + float(db_se.group(14))/60 + float(db_se.group(15))/3600 + float(db_se.group(16))/360000
        else:
          NDB_LAT = 0 - float(db_se.group(13)) - float(db_se.group(14))/60 - float(db_se.group(15))/3600 - float(db_se.group(16))/360000
        if db_se.group(17) == "E":
          NDB_LON = float(db_se.group(18)) + float(db_se.group(19))/60 + float(db_se.group(20))/3600 + float(db_se.group(21))/360000
        else:
          NDB_LON = 0 - float(db_se.group(18)) - float(db_se.group(19))/60 - float(db_se.group(20))/3600 - float(db_se.group(21))/360000
        if db_se.group(22) == "E":
          MAG_VAR = float(db_se.group(23))/10
        else:
          MAG_VAR = 0 - float(db_se.group(23))/10
        NDB_NAME = db_se.group(25).replace('NDB','').rstrip()
        NavaidID = NavaidID + 1
        Navaids_mjc.append((NavaidID,NDB_IDENT,NAVAID_TYPE,NDB_NAME,NDB_FREQ,None,NAVAID_CLASS,NDB_LAT,NDB_LON,0,MAG_VAR))
        NavaidLookup_mjc.append((NDB_IDENT,NAVAID_TYPE,NDB_ICAO_CODE,1,NavaidID))
        Waypoints_mjc.append((NavaidID,NDB_IDENT,1,NDB_NAME,NDB_LAT,NDB_LON,int(NavaidID)))
        WaypointLookup_mjc.append((NDB_IDENT,NDB_ICAO_CODE,NavaidID))
      if ea_se:
        WAYPOINT_IDENT = ea_se.group(8).strip()
        WAYPOINT_ICAO_CODE = ea_se.group(9)
        if ea_se.group(13) == "N":
          WAYPOINT_LAT = round(float(ea_se.group(14)) + float(ea_se.group(15))/60 + float(ea_se.group(16))/3600 + float(ea_se.group(17))/360000, 8)
        else:
          WAYPOINT_LAT = round(0 - float(ea_se.group(14)) - float(ea_se.group(15))/60 - float(ea_se.group(16))/3600 - float(ea_se.group(17))/360000, 8)
        if ea_se.group(18) == "E":
          WAYPOINT_LON = round(float(ea_se.group(19)) + float(ea_se.group(20))/60 + float(ea_se.group(21))/3600 + float(ea_se.group(22))/360000, 8)
        else:
          WAYPOINT_LON = round(0 - float(ea_se.group(19)) - float(ea_se.group(20))/60 - float(ea_se.group(21))/3600 - float(ea_se.group(22))/360000, 8)
        WAYPOINT_NAME_DESCR = ea_se.group(26).rstrip()
        WaypointID = WaypointID + 1
        Waypoints_mjc.append((WaypointID,WAYPOINT_IDENT,0,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,None))
        WaypointLookup_mjc.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID))
      if arpt_se:
        ARPT_IDENT = arpt_se.group(4)
        ARPT_ICAO_CODE = arpt_se.group(5)
        if arpt_se.group(13) == "N":
          ARPT_LAT = round(float(arpt_se.group(14)) + float(arpt_se.group(15))/60 + float(arpt_se.group(16))/3600 + float(arpt_se.group(17))/360000, 8)
        else:
          ARPT_LAT = round(0 - float(arpt_se.group(14)) - float(arpt_se.group(15))/60 - float(arpt_se.group(16))/3600 - float(arpt_se.group(17))/360000, 8)
        if arpt_se.group(18) == "E":
          ARPT_LON = round(float(arpt_se.group(19)) + float(arpt_se.group(20))/60 + float(arpt_se.group(21))/3600 + float(arpt_se.group(22))/360000, 8)
        else:
          ARPT_LON = round(0 - float(arpt_se.group(19)) - float(arpt_se.group(20))/60 - float(arpt_se.group(21))/3600 - float(arpt_se.group(22))/360000, 8)
        ARPT_ELEV = arpt_se.group(25).lstrip('0')
        ARPT_NAME = arpt_se.group(36).rstrip()
        AirportID = AirportID + 1
        Airports_mjc.append((AirportID,ARPT_NAME,ARPT_IDENT,None,ARPT_LAT,ARPT_LON,ARPT_ELEV))
        AirportLookup_mjc.append((ARPT_ICAO_CODE + ARPT_IDENT,AirportID))
      if pc_se:
        WAYPOINT_IDENT = pc_se.group(8).strip()
        WAYPOINT_ICAO_CODE = pc_se.group(9)
        if pc_se.group(13) == "N":
          WAYPOINT_LAT = round(float(pc_se.group(14)) + float(pc_se.group(15))/60 + float(pc_se.group(16))/3600 + float(pc_se.group(17))/360000, 8)
        else:
          WAYPOINT_LAT = round(0 - float(pc_se.group(14)) - float(pc_se.group(15))/60 - float(pc_se.group(16))/3600 - float(pc_se.group(17))/360000, 8)
        if pc_se.group(18) == "E":
          WAYPOINT_LON = round(float(pc_se.group(19)) + float(pc_se.group(20))/60 + float(pc_se.group(21))/3600 + float(pc_se.group(22))/360000, 8)
        else:
          WAYPOINT_LON = round(0 - float(pc_se.group(19)) - float(pc_se.group(20))/60 - float(pc_se.group(21))/3600 - float(pc_se.group(22))/360000, 8)
        WAYPOINT_NAME_DESCR = pc_se.group(26).rstrip()
        WaypointID = WaypointID + 1
        Waypoints_mjc.append((WaypointID,WAYPOINT_IDENT,0,WAYPOINT_NAME_DESCR,WAYPOINT_LAT,WAYPOINT_LON,None))
        WaypointLookup_mjc.append((WAYPOINT_IDENT,WAYPOINT_ICAO_CODE,WaypointID))
    rom = sqlite3.connect('MJC.db3')
    rom_cur = rom.cursor()
    if new_database:
      rom_cur.execute('DROP TABLE IF EXISTS AirportLookup;')
      rom_cur.execute('DROP TABLE IF EXISTS Airports;')
      rom_cur.execute('DROP TABLE IF EXISTS AirwayLegs;')
      rom_cur.execute('DROP TABLE IF EXISTS Airways;')
      rom_cur.execute('DROP TABLE IF EXISTS config;')
      rom_cur.execute('DROP TABLE IF EXISTS ILSes;')
      rom_cur.execute('DROP TABLE IF EXISTS NavaidLookup;')
      rom_cur.execute('DROP TABLE IF EXISTS Navaids;')
      rom_cur.execute('DROP TABLE IF EXISTS NavaidTypes;')
      rom_cur.execute('DROP TABLE IF EXISTS Runways;')
      rom_cur.execute('DROP TABLE IF EXISTS SurfaceTypes;')
      rom_cur.execute('DROP TABLE IF EXISTS TerminalLegs;')
      rom_cur.execute('DROP TABLE IF EXISTS TerminalLegsEx;')
      rom_cur.execute('DROP TABLE IF EXISTS Terminals;')
      rom_cur.execute('DROP TABLE IF EXISTS TrmLegTypes;')
      rom_cur.execute('DROP TABLE IF EXISTS WaypointLookup;')
      rom_cur.execute('DROP TABLE IF EXISTS Waypoints;')
      rom_cur.execute('CREATE TABLE "AirportLookup" ("extID" TEXT, "ID" INTEGER DEFAULT 0 REFERENCES "Airports" ("ID"), PRIMARY KEY("extID","ID"));')
      rom_cur.execute('CREATE TABLE "Airports" ("ID" INTEGER DEFAULT 0, "Name" TEXT, "ICAO" TEXT, "PrimaryID" INTEGER DEFAULT Null REFERENCES "Airports" ("ID"), "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Elevation" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
      rom_cur.execute('CREATE TABLE "AirwayLegs" ("ID" INTEGER DEFAULT 0, "AirwayID" INTEGER DEFAULT 0 REFERENCES "Airways" ("ID"), "Level" TEXT, "Waypoint1ID" INTEGER DEFAULT Null REFERENCES "Waypoints" ("ID"), "Waypoint2ID" INTEGER DEFAULT Null REFERENCES "Waypoints" ("ID"), "IsStart" BOOLEAN NOT NULL, "IsEnd" BOOLEAN NOT NULL, PRIMARY KEY("ID"));')
      rom_cur.execute('CREATE TABLE "Airways" ("ID" INTEGER DEFAULT 0, "Ident" TEXT NOT NULL, PRIMARY KEY("ID"));')
      rom_cur.execute('CREATE TABLE "config" ("key" TEXT, "val" TEXT, PRIMARY KEY("key"));')
      rom_cur.execute('CREATE TABLE "ILSes" ("ID" INTEGER NOT NULL DEFAULT 0, "RunwayID" INTEGER DEFAULT 0 REFERENCES "Runways" ("ID"), "Freq" INTEGER DEFAULT 0, "GsAngle" DOUBLE DEFAULT 0, "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Category" INTEGER DEFAULT 0, "Ident" TEXT, "LocCourse" DOUBLE DEFAULT 0, "CrossingHeight" INTEGER DEFAULT 0, "HasDme" BOOLEAN NOT NULL, "Elevation" INTEGER DEFAULT 0, "LatitudeGS" DOUBLE DEFAULT 0, "LongtitudeGS" DOUBLE DEFAULT 0, "LatitudeDME" DOUBLE DEFAULT 0, "LongtitudeDME" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
      rom_cur.execute('CREATE TABLE "NavaidLookup" ("Ident" TEXT NOT NULL, "Type" INTEGER NOT NULL DEFAULT 0, "Country" TEXT NOT NULL, "NavKeyCode" INTEGER NOT NULL DEFAULT 0, "ID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Navaids" ("ID"), PRIMARY KEY("Ident","Type","Country","NavKeyCode"));')
      rom_cur.execute('CREATE TABLE "Navaids" ("ID" INTEGER DEFAULT 0, "Ident" TEXT, "Type" INTEGER DEFAULT 0 REFERENCES "NavaidTypes" ("Type"), "Name" TEXT, "Freq" INTEGER, "Channel" TEXT, "Usage" TEXT, "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Elevation" INTEGER DEFAULT 0, "SlavedVar" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
      rom_cur.execute('CREATE TABLE "NavaidTypes" ("Type" INTEGER DEFAULT 0, "Desc" TEXT, PRIMARY KEY("Type"));')
      rom_cur.execute('CREATE TABLE "Runways" ("ID" INTEGER DEFAULT 0, "AirportID" INTEGER DEFAULT 0 REFERENCES "Airports" ("ID"), "Ident" TEXT, "TrueHeading" DOUBLE DEFAULT 0, "Length" INTEGER DEFAULT 0, "Width" INTEGER DEFAULT 0, "Surface" TEXT REFERENCES "SurfaceTypes" ("SurfaceType"), "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "Elevation" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
      rom_cur.execute('CREATE TABLE "SurfaceTypes" ("SurfaceType" TEXT, "Description" TEXT, PRIMARY KEY("SurfaceType"));')
      rom_cur.execute('CREATE TABLE "TerminalLegs" ("ID" INTEGER DEFAULT 0 REFERENCES "TerminalLegsEx" ("ID"), "TerminalID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Terminals" ("ID"), "Type" TEXT, "Transition" TEXT, "TrackCode" TEXT REFERENCES "TrmLegTypes" ("Code"), "WptID" INTEGER DEFAULT 0 REFERENCES "Waypoints" ("ID"), "WptLat" DOUBLE DEFAULT 0, "WptLon" DOUBLE DEFAULT 0, "TurnDir" TEXT, "NavID" INTEGER DEFAULT 0 REFERENCES "Navaids" ("ID"), "NavLat" DOUBLE DEFAULT 0, "NavLon" DOUBLE DEFAULT 0, "NavBear" DOUBLE DEFAULT 0, "NavDist" DOUBLE DEFAULT 0, "Course" DOUBLE DEFAULT 0, "Distance" DOUBLE DEFAULT 0, "Alt" TEXT, "Vnav" DOUBLE DEFAULT 0, "CenterID" INTEGER DEFAULT 0, "CenterLat" DOUBLE DEFAULT 0, "CenterLon" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
      rom_cur.execute('CREATE TABLE "TerminalLegsEx" ("ID" INTEGER DEFAULT 0, "IsFlyOver" BOOLEAN NOT NULL, "SpeedLimit" DOUBLE DEFAULT 0, PRIMARY KEY("ID"));')
      rom_cur.execute('CREATE TABLE "Terminals" ("ID" INTEGER DEFAULT 0, "AirportID" INTEGER DEFAULT 0 REFERENCES "Airports" ("ID"), "Proc" INTEGER DEFAULT 0, "ICAO" TEXT NOT NULL, "FullName" TEXT NOT NULL, "Name" TEXT NOT NULL, "Rwy" TEXT, "RwyID" INTEGER DEFAULT 0, PRIMARY KEY("ID"));')
      rom_cur.execute('CREATE TABLE "TrmLegTypes" ("Code" TEXT NOT NULL, "Description" TEXT, PRIMARY KEY("Code"));')
      rom_cur.execute('CREATE TABLE "WaypointLookup" ("Ident" TEXT NOT NULL, "Country" TEXT NOT NULL, "ID" INTEGER NOT NULL DEFAULT 0 REFERENCES "Waypoints" ("ID"), PRIMARY KEY("Ident","Country","ID"));')
      rom_cur.execute('CREATE TABLE "Waypoints" ("ID" INTEGER DEFAULT 0, "Ident" TEXT NOT NULL, "Collocated" BOOLEAN NOT NULL, "Name" TEXT, "Latitude" DOUBLE DEFAULT 0, "Longtitude" DOUBLE DEFAULT 0, "NavaidID" INTEGER DEFAULT Null REFERENCES "Navaids" ("ID"), PRIMARY KEY("ID"));')
    if write_database:
      AirportLookup_mjc_tb = pandas.DataFrame(AirportLookup_mjc, columns=['extID','ID'])
      Airports_mjc_tb = pandas.DataFrame(Airports_mjc, columns=['ID','Name','ICAO','PrimaryID','Latitude','Longtitude','Elevation'])
      AirwayLegs_mjc_tb = pandas.DataFrame(AirwayLegs_mjc, columns=['ID','AirwayID','Level','Waypoint1ID','Waypoint2ID','IsStart','IsEnd'])
      Airways_mjc_tb = pandas.DataFrame(Airways_mjc, columns=['ID','Ident'])
      ILSes_mjc_tb = pandas.DataFrame(ILSes_mjc, columns=['ID','RunwayID','Freq','GsAngle','Latitude','Longtitude','Category','Ident','LocCourse','CrossingHeight','HasDme','Elevation','LatitudeGS','LongtitudeGS','LatitudeDME','LongtitudeDME'])
      NavaidLookup_mjc_tb = pandas.DataFrame(NavaidLookup_mjc, columns=['Ident','Type','Country','NavKeyCode','ID'])
      Navaids_mjc_tb = pandas.DataFrame(Navaids_mjc, columns=['ID','Ident','Type','Name','Freq','Channel','Usage','Latitude','Longtitude','Elevation','SlavedVar'])
      Runways_mjc_tb = pandas.DataFrame(Runways_mjc, columns=['ID','AirportID','Ident','TrueHeading','Length','Width','Surface','Latitude','Longtitude','Elevation'])
      TerminalLegs_mjc_tb = pandas.DataFrame(TerminalLegs_mjc, columns=['ID','TerminalID','Type','Transition','TrackCode','WptID','WptLat','WptLon','TurnDir','NavID','NavLat','NavLon','NavBear','NavDist','Course','Distance','Alt','Vnav','CenterID','CenterLat','CenterLon'])
      TerminalLegsEx_mjc_tb = pandas.DataFrame(TerminalLegsEx_mjc, columns=['ID','IsFlyOver','SpeedLimit'])
      Terminals_mjc_tb = pandas.DataFrame(Terminals_mjc, columns=['ID','AirportID','Proc','ICAO','FullName','Name','Rwy','RwyID'])
      WaypointLookup_mjc_tb = pandas.DataFrame(WaypointLookup_mjc, columns=['Ident','Country','ID'])
      Waypoints_mjc_tb = pandas.DataFrame(Waypoints_mjc, columns=['ID','Ident','Collocated','Name','Latitude','Longtitude','NavaidID'])
      AirportLookup_mjc_tb.to_sql('AirportLookup', rom, if_exists='append', index=False)
      Airports_mjc_tb.to_sql('Airports', rom, if_exists='append', index=False)
      AirwayLegs_mjc_tb.to_sql('AirwayLegs', rom, if_exists='append', index=False)
      Airways_mjc_tb.to_sql('Airways', rom, if_exists='append', index=False)
      ILSes_mjc_tb.to_sql('ILSes', rom, if_exists='append', index=False)
      NavaidLookup_mjc_tb.to_sql('NavaidLookup', rom, if_exists='append', index=False)
      Navaids_mjc_tb.to_sql('Navaids', rom, if_exists='append', index=False)
      Runways_mjc_tb.to_sql('Runways', rom, if_exists='append', index=False)
      TerminalLegs_mjc_tb.to_sql('TerminalLegs', rom, if_exists='append', index=False)
      TerminalLegsEx_mjc_tb.to_sql('TerminalLegsEx', rom, if_exists='append', index=False)
      Terminals_mjc_tb.to_sql('Terminals', rom, if_exists='append', index=False)
      WaypointLookup_mjc_tb.to_sql('WaypointLookup', rom, if_exists='append', index=False)
      Waypoints_mjc_tb.to_sql('Waypoints', rom, if_exists='append', index=False)
    if sinonav:
      a = 0
    if add_tailored:
      a = 0
    Airports_mjc_tb.to_csv(path_or_buf='MJC-ARPT.txt', sep='|', header=False, index=False)
    AirportLookup_mjc_tb.to_csv(path_or_buf='MJC-ARPT_Lookup.txt', sep='|', header=False, index=False)
    Navaids_mjc_tb.to_csv(path_or_buf='MJC-NAVAID.txt', sep='|', header=False, index=False)
    NavaidLookup_mjc_tb.to_csv(path_or_buf='MJC-NAVAID_Lookup.txt', sep='|', header=False, index=False)
    Waypoints_mjc_tb.to_csv(path_or_buf='MJC-WPT.txt', sep='|', header=False, index=False)
    WaypointLookup_mjc_tb.to_csv(path_or_buf='MJC-WPT_Lookup.txt', sep='|', header=False, index=False)
    rom_cur.close()
    rom.commit()
    rom.close()
